## Download ENTSOE-e data

In [ ]:
"""
Download every 2025*.zip found inside each subfolder of /TP_export/zip
and save it locally under the same subfolder name.
"""

import paramiko
import os
import stat
from pathlib import Path
from dotenv import load_dotenv # Added import

HOST = "sftp-transparency.entsoe.eu"
PORT = 22
# Updated to load from environment variables
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
REMOTE_BASE = "/TP_export/zip"
LOCAL_BASE = Path("TP_export_zip")


def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def main() -> None:
    load_dotenv() # Load .env file
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # auto-trust host key once
    ssh.connect(HOST, port=PORT, username=USERNAME, password=PASSWORD)

    sftp = ssh.open_sftp()

    for entry in sftp.listdir_attr(REMOTE_BASE):
        if stat.S_ISDIR(entry.st_mode):
            remote_dir = f"{REMOTE_BASE}/{entry.filename}"
            local_dir = os.path.join(LOCAL_BASE, entry.filename)
            ensure_dir(local_dir)

            for file_attr in sftp.listdir_attr(remote_dir):
                name = file_attr.filename
                if name.startswith("2025") and name.endswith(".zip"):
                    remote_file = f"{remote_dir}/{name}"
                    local_file = os.path.join(local_dir, name)
                    print(f"⬇︎ {remote_file} → {local_file}")
                    sftp.get(remote_file, local_file)

    sftp.close()
    ssh.close()
    print("Done!")

if __name__ == "__main__":
    main()

## Unzip and sort

In [ ]:
import os
import zipfile
import pandas as pd
import concurrent.futures
from pathlib import Path
from typing import List, Tuple
import pyarrow as pa
import pyarrow.csv as csv
from functools import partial

def extract_from_zip(zip_file: Path, folder_path: Path) -> List[str]:
    """Extract CSV files from a ZIP file and return list of extracted filenames."""
    extracted_files = []
    try:
        with zipfile.ZipFile(zip_file, 'r') as z:
            for member in z.namelist():
                if member.lower().endswith(".csv"):
                    z.extract(member, path=folder_path)
                    extracted_files.append(member)
                    print(f"Extracted {member} from {zip_file.name}")
    except Exception as e:
        print(f"Error processing zip file {zip_file.name}: {e}")
    return extracted_files

def process_csv_chunk(csv_file: Path) -> Tuple[pd.DataFrame, str]:
    """Process a single CSV file handling both comma and tab separators."""
    try:
        # Try reading with tab separator first
        try:
            df = pd.read_csv(csv_file, sep='\t', engine='python')
        except:
            # If tab separator fails, try comma separator
            df = pd.read_csv(csv_file, sep=',', engine='python')
        
        # Clean column names by stripping whitespace
        df.columns = df.columns.str.strip()
        return df, csv_file.name
    except Exception as e:
        print(f"Error processing {csv_file.name}: {e}")
        return pd.DataFrame(), csv_file.name

def process_folder(folder_path: Path):
    """Optimized version of process_folder using parallel processing and efficient I/O."""
    print(f"\nProcessing folder: {folder_path.name}")
    
    # Look for CSV files in the folder
    csv_files = list(folder_path.glob("*.csv"))
    
    # Handle ZIP files if no CSVs found
    if not csv_files:
        zip_files = list(folder_path.glob("*.zip"))
        if zip_files:
            print("No CSV files found. Checking ZIP files for CSV extraction...")
            # Process ZIP files in parallel
            with concurrent.futures.ThreadPoolExecutor() as executor:
                futures = [
                    executor.submit(extract_from_zip, zip_file, folder_path)
                    for zip_file in zip_files
                ]
                concurrent.futures.wait(futures)
            
            # Remove ZIP files after extraction
            for zip_file in zip_files:
                try:
                    os.remove(zip_file)
                    print(f"Removed zip file: {zip_file.name}")
                except Exception as e:
                    print(f"Failed to remove zip file {zip_file.name}: {e}")
            
            # Update CSV files list
            csv_files = list(folder_path.glob("*.csv"))
        else:
            print("No CSV or ZIP files found in folder, skipping processing.")
            return

    if not csv_files:
        print("No CSV files could be found or extracted, skipping processing.")
        return

    # Process CSV files in parallel using ThreadPoolExecutor
    dfs = []
    csv_names = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_csv = {
            executor.submit(process_csv_chunk, csv_file): csv_file
            for csv_file in csv_files
        }
        for future in concurrent.futures.as_completed(future_to_csv):
            df, name = future.result()
            if not df.empty:
                dfs.append(df)
                csv_names.append(name)

    if not dfs:
        print("No valid CSV data was found, skipping processing.")
        return

    # Combine all DataFrames efficiently
    combined_df = pd.concat(dfs, ignore_index=True, copy=False)

    # Sort by date if date column exists
    date_cols = [col for col in combined_df.columns if 'date' in col.lower()]
    if date_cols:
        combined_df.sort_values(by=date_cols[0], inplace=True, ignore_index=True)

    # Prepare output directories
    processed_dir = Path("processed_data")
    filtered_dir = Path("filtered_data")
    processed_dir.mkdir(exist_ok=True)
    filtered_dir.mkdir(exist_ok=True)

    # Save combined CSV efficiently using pyarrow
    combined_csv_path = processed_dir / f"{folder_path.name}.csv"
    try:
        table = pa.Table.from_pandas(combined_df)
        csv.write_csv(table, combined_csv_path)
        print(f"Combined CSV saved to: {combined_csv_path}")
    except Exception as e:
        print(f"Failed to write combined CSV: {e}")
        return

    # Create log details
    details = (
        f"Combined CSV file from folder: {folder_path.name}\n"
        f"Files combined: {', '.join(csv_names)}\n"
        f"Total rows: {len(combined_df)}\n"
    )

    # Remove original CSVs and create log file
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(os.remove, csv_files)
    
    log_path = folder_path / f"{folder_path.name}.txt"
    with open(log_path, "w") as log_file:
        log_file.write(details)
    print(f"Log file created at: {log_path}")

    # Handle MapCode filtering or
    # OutAreaMapCode and InAreaMapCode 
    # OutMapCode and InMapCode
    # Check for any of the possible map code columns
    possible_columns = ['mapcode', 'outareamapcode', 'inaremapcode', 'outmapcode', 'inmapcode', 'areamapcode', 'location']
    map_columns = [col for col in combined_df.columns if col.strip().lower() in possible_columns]
    
    if map_columns:
        try:
            # Create a mask that checks for 'SE' in any of the map code columns
            mask = pd.Series(False, index=combined_df.index)
            for col in map_columns:
                mask |= combined_df[col].astype(str).str.contains("SE", na=False, case=False)
            
            # Apply the mask to filter rows
            filtered_df = combined_df[mask]
            filtered_csv_path = filtered_dir / f"{folder_path.name}_SE.csv"
            
            # Save filtered data efficiently using pyarrow
            filtered_table = pa.Table.from_pandas(filtered_df)
            csv.write_csv(filtered_table, filtered_csv_path)
            print(f"Filtered CSV saved to: {filtered_csv_path}")
            
            filter_details = (
                f"Filtering successful for folder: {folder_path.name}\n"
                f"Columns used for filtering: {', '.join(map_columns)}\n"
                f"Filtered CSV (contains 'SE') rows: {len(filtered_df)}\n"
                f"Files combined: {', '.join(csv_names)}\n"
            )
            
            # Clean up and create logs
            os.remove(combined_csv_path)
            filtering_log = combined_csv_path.with_suffix(".txt")
            with open(filtering_log, "w") as f:
                f.write(filter_details)
            print(f"Combined CSV replaced with filtering log: {filtering_log}")
            
        except Exception as e:
            failure_details = (
                f"Filtering failed for folder: {folder_path.name}\n"
                f"Error: {e}\n"
                f"Files combined: {', '.join(csv_names)}\n"
                f"Total rows in combined CSV: {len(combined_df)}\n"
            )
            filtering_log = combined_csv_path.with_suffix(".txt")
            with open(filtering_log, "w") as f:
                f.write(failure_details)
            print(f"Filtering failed; added failure log: {filtering_log}")
    else:
        no_filter_details = (
            f"No MapCode columns found in combined CSV for folder: {folder_path.name}\n"
            f"Checked for columns: {', '.join(possible_columns)}\n"
            f"Available columns: {', '.join(combined_df.columns)}\n"
            f"Filtering was not applied.\n"
        )
        filtering_log = combined_csv_path.with_suffix(".txt")
        with open(filtering_log, "w") as f:
            f.write(no_filter_details)
        print(f"No MapCode columns; filtering log created: {filtering_log}")

def main():
    base_dir = Path("TP_export_zip")
    if not base_dir.exists():
        print(f"Base directory '{base_dir}' does not exist.")
    else:
        folders = [folder for folder in base_dir.iterdir() if folder.is_dir()]
        # Process folders sequentially but with internal parallelization
        for folder in folders:
            process_folder(folder)

if __name__ == "__main__":
    main()

## Move into appropiate folder

In [ ]:

from pathlib import Path
import re
import shutil

SECTION_TO_FOLDER = {
    "17": "Balancing",
    "12.3": "Balancing",          # special Balancing subsection
    "13": "Congestion Management",
    "16": "Generation",
    "14": "Generation",
    "6":  "Load",
    "8.1": "Load",
    "7":  "Outages",
    "10": "Outages",
    "15": "Outages",
    "11": "Transmission",
    "12.1": "Transmission",
}

# pattern:  <anything>_<digits[.digits][.letter]>  e.g. _17.1.D or _6.1 or _12.3.E
CODE_RX = re.compile(r"_(\d+(?:\.\d+)?)(?:\.[A-Z])?_")   # group(1) = 17, 12.3, 6 …

HERE = Path("filtered_data")

moved, skipped = 0, 0
for csv in HERE.glob("*.csv"):
    m = CODE_RX.search(csv.name)
    if not m:
        skipped += 1
        print(f"couldnt recognise code in {csv.name}; leaving it in place")
        continue

    section = m.group(1)          # e.g. '17', '12.3', '6', '8.1'
    # pick the longest matching key (so 12.3 wins over 12)
    dest_folder = next(
        (SECTION_TO_FOLDER[k] for k in sorted(SECTION_TO_FOLDER, key=len, reverse=True)
         if section.startswith(k)),
        None,
    )

    if dest_folder is None:
        skipped += 1
        print(f"no rule for section {section} → {csv.name}")
        continue

    dest_dir = HERE / dest_folder
    dest_dir.mkdir(exist_ok=True)
    shutil.move(str(csv), dest_dir / csv.name)
    moved += 1
    print(f"{csv.name}  →  {dest_folder}/")

print(f"\nDone — moved {moved} file(s), skipped {skipped}.")

## Proccess long to wide format

### Balancing folder


In [ ]:
import pandas as pd
import os
import json
from pathlib import Path

# Configuration for each file
config = [
    {
        "Balancing folder": "AcceptedAggregatedOffers_17.1.D_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode, ReserveType",
        "Keep columns": "ResolutionCode, AreaTypeCode, NotSpecifiedUpAcceptedVolume, NotSpecifiedDownAcceptedVolume, NotSpecifiedOfferedVolumeSymmetric, NotSpecifiedAcceptedVolumeSymmetric"
    },
    {
        "Balancing folder": "ActivatedBalancingEnergy_17.1.E_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode, ReserveType",
        "Keep columns": "ResolutionCode, AreaTypeCode, NotSpecifiedUpActivatedVolume, NotSpecifiedDownActivatedVolume"
    },
    {
        "Balancing folder": "AggregatedBalancingEnergyBids_12.3.E_r3_SE.csv",
        "Index/date column name": "DateTime(UTC)",
        "Merging column": "MapCode, ReserveType",
        "Keep columns": "ResolutionCode, AreaTypeCode, OfferedUpBidVolume[MW], OfferedDownBidVolume[MW], ActivatedDownBidVolume[MW], ActivatedUpBidVolume[MW]"
    },
    {
        "Balancing folder": "ImbalancePrices_17.1.G_r2.1_SE.csv",
        "Index/date column name": "DateTime(UTC)",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, PositiveImbalancePrice, NegativeImbalancePrice"
    },
    {
        "Balancing folder": "PricesOfActivatedBalancingEnergy_17.1.F_r3_SE.csv",
        "Index/date column name": "ISP(UTC)",
        "Merging column": "MapCode, ReserveType",
        "Keep columns": "ResolutionCode, AreaTypeCode, TypeOfProduct, GenerationUpPrice, GenerationDownPrice, NotSpecifiedUpPrice, NotSpecifiedDownPrice, Currency"
    },
    # {
    #     "Balancing folder": "PricesOfProcuredBalancingReserves_17.1.C_SE.csv",
    #     "Index/date column name": "DateTime",
    #     "Merging column": "MapCode, ReserveType, Direction",
    #     "Keep columns": "ResolutionCode, AreaTypeCode, ContractedPrice"
    # },
    {
        "Balancing folder": "TotalimbalanceVolumes_17.1.H_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, TotalImbalanceVolume"
    },
    {
        "Balancing folder": "FinancialExpensesAndincomeForBalancing_17.1.I_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, Expenses, Income"
    },
]


def process_file(file_path, file_config):
    """
    Process a single file to convert from long format to wide format
    based on the specified configuration.
    """
    try:
        df = pd.read_csv(file_path, low_memory=False)
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            return None
    
    date_col = file_config["Index/date column name"]
    
    # Get configured ID columns and value columns
    merging_cols_from_config_str = file_config["Merging column"]
    # Handle if merging column string is empty or just spaces
    if not merging_cols_from_config_str.strip():
        merging_cols_list_config = []
    else:
        merging_cols_list_config = [col.strip() for col in merging_cols_from_config_str.split(",")]
        
    keep_cols_list_config = [col.strip() for col in file_config["Keep columns"].split(",")]

    # Filter these to only include columns that actually exist in the DataFrame
    # These id_cols and value_cols are the ones that will be used.
    id_cols = [col for col in merging_cols_list_config if col in df.columns]
    value_cols = [col for col in keep_cols_list_config if col in df.columns]

    # Check if date_col exists (critical for indexing and pivot)
    if date_col not in df.columns:
        print(f"Critical: Date column '{date_col}' missing in {file_path}. Skipping file.")
        return None

    # Determine the minimal set of columns to keep in the initial DataFrame
    # This includes the date column, all (existing) ID columns, and all (existing) value columns.
    # Using a set to avoid duplicates, then converting to list to preserve order from original df.columns (if possible) or a defined order.
    actual_cols_to_load = {date_col}
    actual_cols_to_load.update(id_cols)
    actual_cols_to_load.update(value_cols)
    
    # Filter df to only these necessary columns, preserving original order as much as possible
    cols_present_in_df = [col for col in df.columns if col in actual_cols_to_load]
    df = df[cols_present_in_df]
    
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # 1. build the ID safely (as per user's patch structure)
    # id_cols here is the filtered list of *existing* merging column names.
    if id_cols:
        # turn each row into "_".join([...]) → always a scalar string
        df["id"] = (
            df[id_cols]
            .fillna("")                        # don't drop rows because of NaNs
            .astype(str)
            .apply(lambda r: "_".join(r).strip("_"), axis=1)
        )
    else:                                      # no merging columns configured/found
        df["id"] = "value"

    # 2. initialise the result index once (as per user's patch structure)
    # df now contains 'id' (if id_cols was not empty) and 'date_col'
    # Ensure date_col is still present (should be, due to earlier check)
    # And 'id' column is also used by pivot_table if id_cols was present.
    if date_col not in df.columns: # Should not happen if initial check passed
         print(f"Critical error: date_col '{date_col}' lost before result initialization for {file_path}.")
         return None
    
    # Create the base result DataFrame with a unique sorted datetime index
    # If 'id' column was not created (because id_cols was empty), pivot_table will use 'value' as id.
    result_index_df = df[[date_col]].copy() # Operate on a copy to avoid SettingWithCopyWarning
    result_index_df.drop_duplicates(subset=[date_col], inplace=True)
    result_index_df.set_index(date_col, inplace=True)
    result_index_df.sort_index(inplace=True)
    result = result_index_df 
    # result.index = pd.to_datetime(result.index) # Index is already datetime

    # 3. wide-format one value column at a time (as per user's patch structure)
    # value_cols is the filtered list of *existing* value column names.
    for col in value_cols:
        if col not in df.columns: # Should not happen if value_cols are derived from df.columns
            print(f"Warning: Value column '{col}' unexpectedly missing in df for {file_path}. Skipping.")
            continue
        if 'id' not in df.columns: # Check if 'id' column exists for pivot
             print(f"Critical error: 'id' column missing before pivot for {file_path} (value col: {col}).")
             continue


        pivot = df.pivot_table(
            index=date_col,
            columns="id", # Uses the 'id' column created above
            values=col,   # Current value column to pivot
            aggfunc="first" # Aggregation function for duplicates
        )
        
        # Rename columns: original_id_value_CellValueColumnName
        pivot.columns = [f"{id_val}_{col}" for id_val in pivot.columns]
        
        result = result.join(pivot) # Join pivoted data to the main result DataFrame
    
    # Filter out columns containing "_SEM_" (remains from original script logic)
    sem_columns = [col for col in result.columns if "_SEM_" in col]
    if sem_columns:
        print(f"  Removing {len(sem_columns)} columns containing '_SEM_'")
        result = result.drop(columns=sem_columns)
    
    if result.empty and not value_cols:
        print(f"  Processed {file_path}. No value columns were specified or found, result is empty based on index.")
    elif result.empty and value_cols :
        print(f"  Processed {file_path}. Resulting DataFrame is empty despite value columns being present. Check data and pivoting logic.")
    else:
        print(f"  Processed {file_path} into {result.shape[1]} columns")
    return result


def main():
    input_folder_path = Path("filtered_data/Balancing")
    output_folder_path = Path("pivoted_data/Balancing")
    os.makedirs(output_folder_path, exist_ok=True)
    
    for file_config in config:
        file_name = file_config["Balancing folder"]
        file_path = input_folder_path / file_name # Use Path object for joining
        
        if file_path.exists():
            print(f"Processing {file_name}...")
            result_df = process_file(str(file_path), file_config) # process_file expects string path
            
            if result_df is not None and not result_df.empty:
                base_name = file_path.stem # Get filename without extension
                output_file = f"{base_name}_pivoted.csv"
                output_path = output_folder_path / output_file
                
                result_df.to_csv(output_path)
                
                print(f"  Saved to {output_path}")
                print(f"  Shape: {result_df.shape}")
                if not result_df.index.empty:
                    print(f"  Date range: {result_df.index.min()} to {result_df.index.max()}")
                else:
                    print(f"  Date range: Index is empty.")
                if result_df.size > 0 :
                    missing_percentage = 100 * result_df.isna().sum().sum() / result_df.size
                    print(f"  Missing values: {result_df.isna().sum().sum()}/{result_df.size} ({missing_percentage:.2f}%)")
                else:
                    print(f"  Missing values: DataFrame is empty, no values.")
                print(f"  Sample columns: {list(result_df.columns)[:min(3, len(result_df.columns))]}")
                print("")
            elif result_df is not None and result_df.empty:
                print(f"  Processed {file_name}, but the resulting DataFrame is empty. No file saved.")
            else: # result_df is None
                print(f"  Failed to process {file_name} or an error occurred.")
        else:
            print(f"File not found: {file_path}")
    
    print("\nAll files processed.")
    print(f"Output folder: {output_folder_path.resolve()}")

if __name__ == "__main__":
    main()

In [ ]:
from pathlib import Path
import pandas as pd
import os

# Configuration for each file
config = [
    {
        "Balancing folder": "AmountOfBalancingReservesUnderContract_17.1.B_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode, ReserveType, Direction",
        "Keep columns": "ResolutionCode, AreaTypeCode, ContractedVolume, SourceName, ContractType",
        "Aggregation methods": {
            "ContractedVolume": "sum"
        }
    },
]

def process_file(file_path, file_config):
    """
    Process a single file to convert from long format to wide format
    based on the specified configuration, handling duplicate entries.
    """
    # Read CSV (UTF-8 fallback to latin1)
    try:
        df = pd.read_csv(file_path, low_memory=False)
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='latin1', low_memory=False)

    # Extract configuration
    date_col = file_config["Index/date column name"]
    id_cols = [col.strip() for col in file_config["Merging column"].split(",")]
    value_cols = [col.strip() for col in file_config["Keep columns"].split(",")]
    agg_methods = file_config.get("Aggregation methods", {})

    # Warn if any columns are missing
    missing = set(id_cols + [date_col] + value_cols) - set(df.columns)
    if missing:
        print(f"Warning: Missing columns in {file_path}: {missing}")
        id_cols = [c for c in id_cols if c in df.columns]
        value_cols = [c for c in value_cols if c in df.columns]

    # Convert date column
    df[date_col] = pd.to_datetime(df[date_col])

    # Optional duplicate check
    if id_cols:
        dupes = df.groupby([date_col] + id_cols).size()
        dup_count = (dupes > 1).sum()
        if dup_count > 0:
            print(f"  ⚠️ {dup_count} duplicate date+ID combinations found")

    # Prepare result DataFrame indexed by all timestamps
    timestamps = sorted(df[date_col].unique())
    result = pd.DataFrame(index=timestamps)
    result.index.name = date_col

    # Pivot each value column
    for col in value_cols:
        method = agg_methods.get(col, 'first')
        pivot = df.pivot_table(
            index=date_col,
            columns=id_cols if id_cols else None,
            values=col,
            aggfunc=method
        )
        # Flatten MultiIndex or single index
        if isinstance(pivot.columns, pd.MultiIndex):
            pivot.columns = ["_".join(map(str, key)) + f"_{col}" for key in pivot.columns]
        else:
            pivot.columns = [str(key) + f"_{col}" for key in pivot.columns]
        result = result.join(pivot)

    # Remove any '_SEM_' columns
    sem_cols = [c for c in result.columns if '_SEM_' in c]
    if sem_cols:
        result.drop(columns=sem_cols, inplace=True)

    print(f"  Processed {file_path}: {result.shape[1]} columns")
    return result


def main():
    input_folder = Path("filtered_data/Balancing")
    output_folder = Path("pivoted_data/Balancing")
    output_folder.mkdir(parents=True, exist_ok=True)

    for cfg in config:
        fname = cfg["Balancing folder"]
        in_path = input_folder / fname
        if not in_path.exists():
            print(f"File not found: {in_path}")
            continue

        print(f"Processing {fname}...")
        df_wide = process_file(in_path, cfg)
        if df_wide is None:
            continue

        out_name = in_path.stem + '_pivoted.csv'
        out_path = output_folder / out_name
        df_wide.to_csv(out_path)

        # Report
        total = df_wide.size
        missing = df_wide.isna().sum().sum()
        print(f"  Saved to {out_path}")
        print(f"  Shape: {df_wide.shape}")
        print(f"  Date range: {df_wide.index.min()} to {df_wide.index.max()}")
        print(f"  Missing values: {missing}/{total} ({100*missing/total:.2f}%)")
        print("")

    print("All files processed.")

if __name__ == "__main__":
    main()

### Congestion Management

In [ ]:
import pandas as pd
import os
import json


config = [
    {
        "Congestion Management folder": "Countertrading_13.1.B_r2.1_SE.csv",
        "Index/date column name": "StartTimeSeries(UTC)",
        # "Merging column": "OutAreaMapCode, InAreaMapCode, ",
        "Keep columns": "Resolution, ChangeInCrosszonalExchange(MW), Action"
    },
    {
        "Congestion Management folder": "RedispatchingInternal_13.1.A_r2.1_SE.csv",
        "Index/date column name": "StartTimeSeries(UTC)",
        # "Merging column": "OutAreaMapCode, InAreaMapCode, ",
        "Keep columns": "CapacityImpact(MWh/MTU), ActionDirection(MW)"
    }
    

]


def process_file(file_path, file_config):
    """
    Process a single file to convert from long format to wide format
    based on the specified configuration with special handling for SE regions.
    """
    try:
        # Try reading with UTF-8
        df = pd.read_csv(file_path, low_memory=False)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try 'latin1'
            df = pd.read_csv(file_path, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            return None
    
    # Extract configuration
    date_col = file_config["Index/date column name"]
    value_cols = [col.strip() for col in file_config["Keep columns"].split(",")]
    
    # Check if required columns exist
    required_cols = [date_col, "OutAreaMapCode", "InAreaMapCode"] + value_cols
    missing_cols = set(required_cols) - set(df.columns)
    if missing_cols:
        print(f"Warning: Missing columns in {file_path}: {missing_cols}")
        return None
    
    # Only keep needed columns
    cols_to_keep = [date_col, "OutAreaMapCode", "InAreaMapCode"] + value_cols
    df = df[cols_to_keep]
    
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Define Swedish bidding zones
    swedish_zones = ['SE', 'SE1', 'SE2', 'SE3', 'SE4']
    
    # Filter for rows where either OutAreaMapCode or InAreaMapCode is a Swedish zone
    df_se = df[(df['OutAreaMapCode'].isin(swedish_zones)) | (df['InAreaMapCode'].isin(swedish_zones))]
    
    # Create a direction-based identifier
    df_se['flow_id'] = df_se.apply(
        lambda row: f"FROM_{row['OutAreaMapCode']}_TO_{row['InAreaMapCode']}", 
        axis=1
    )
    
    # Check for duplicates with the new identifier
    duplicate_check = df_se.duplicated(subset=[date_col, 'flow_id'], keep=False)
    if duplicate_check.any():
        dup_count = duplicate_check.sum()
        print(f"  Warning: Found {dup_count} duplicate entries with the same date and flow ID")
        
        # Detect numeric and non-numeric columns
        numeric_cols = []
        non_numeric_cols = []
        
        for col in value_cols:
            try:
                pd.to_numeric(df_se[col])
                numeric_cols.append(col)
            except (ValueError, TypeError):
                non_numeric_cols.append(col)
        
        # Create an aggregation dictionary
        agg_dict = {}
        for col in numeric_cols:
            agg_dict[col] = 'mean'
        for col in non_numeric_cols:
            agg_dict[col] = 'first'
        
        # Apply the appropriate aggregation to each column
        print("  Aggregating duplicate values with mixed methods (mean for numeric, first for non-numeric)")
        df_se = df_se.groupby([date_col, 'flow_id']).agg(agg_dict).reset_index()
    
    # Create a wide-format dataframe
    result = pd.DataFrame(index=df_se[date_col].unique())
    result.index = pd.to_datetime(result.index)
    result.sort_index(inplace=True)
    result.index.name = date_col
    
    # Process each value column
    for col in value_cols:
        # Pivot to get values for each flow ID
        pivot = df_se.pivot(index=date_col, columns='flow_id', values=col)
        
        # Rename columns with a more descriptive format
        pivot.columns = [f"{flow_id}_{col}" for flow_id in pivot.columns]
        
        # Join with the result
        result = result.join(pivot)
    
    print(f"  Processed {file_path} into {result.shape[1]} columns")
    return result

def main():
    # Main processing logic
    # input_folder_path = "../../processed_data/grouped_data/Congestion Management"
    # output_folder_path = "../../processed_data/pivoted_data/Congestion Management"
    input_folder_path = Path("filtered_data/Congestion Management")
    output_folder_path = Path("pivoted_data/Congestion Management")
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Process each file according to its configuration
    for file_config in config:
        file_name = file_config["Congestion Management folder"]
        file_path = os.path.join(input_folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            result_df = process_file(file_path, file_config)
            
            if result_df is not None:
                # Generate output filename - replace the original extension with '_pivoted.csv'
                base_name = os.path.splitext(file_name)[0]
                output_file = f"{base_name}_pivoted.csv"
                output_path = os.path.join(output_folder_path, output_file)
                
                # Save individual pivoted file
                result_df.to_csv(output_path)
                
                # Print statistics for this file
                print(f"  Saved to {output_path}")
                print(f"  Shape: {result_df.shape}")
                print(f"  Date range: {result_df.index.min()} to {result_df.index.max()}")
                print(f"  Missing values: {result_df.isna().sum().sum()}/{result_df.size} " 
                     f"({100*result_df.isna().sum().sum()/result_df.size:.2f}%)")
                print(f"  Sample columns: {list(result_df.columns)[:3]}")
                print("")
        else:
            print(f"File not found: {file_path}")
    
    print("\nAll files processed and saved to individual pivoted CSVs in:")
    print(output_folder_path)

if __name__ == "__main__":
    main()

### Generation

In [ ]:
import pandas as pd
import os
import json

config = [
    # {
    #     "Generation folder": "ActualGenerationOutputPerGenerationUnit_16.1.A_r2.1_SE.csv",
    #     "Index/date column name": "DateTime (UTC)",
    #     "Merging column": "MapCode, GenerationUnitName",
    #     "Keep columns": "ResolutionCode, AreaTypeCode, ActualGenerationOutput(MW), GenerationUnitInstalledCapacity(MW)"
    # },
    {
        "Generation folder": "AggregatedFillingRateOfWaterReservoirsAndHydroStoragePlants_16.1.D_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, StoredEnergy"
    },
    {
        "Generation folder": "AggregatedGenerationPerType_16.1.B_C_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, ProductionType, ActualGenerationOutput"
    },
    {
        "Generation folder": "CurrentGenerationForecastForWindAndSolar_14.1.D_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, ProductionType, AggregatedGenerationForecast"
    },
    {
        "Generation folder": "DayAheadAggregatedGeneration_14.1.C_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, ScheduledGeneration"
    },
    {
        "Generation folder": "DayAheadGenerationForecastForWindAndSolar_14.1.D_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, ProductionType, AggregatedGenerationForecast"
    },
    {
        "Generation folder": "InstalledGenerationCapacityAggregated_14.1.A_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, ProductionType, AggregatedInstalledCapacity"
    },
    {
        "Generation folder": "IntradayGenerationForecastForWindAndSolar_14.1.D_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, ProductionType, AggregatedGenerationForecast"
    }
    

]



def process_file(file_path, file_config):
    """
    Process a single file to convert from long format to wide format
    based on the specified configuration.
    """
    try:
        # Try reading with UTF-8
        df = pd.read_csv(file_path, low_memory=False)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try 'latin1'
            df = pd.read_csv(file_path, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            return None
    
    # Extract configuration
    date_col = file_config["Index/date column name"]
    id_cols = [col.strip() for col in file_config["Merging column"].split(",")]
    value_cols = [col.strip() for col in file_config["Keep columns"].split(",")]
    
    # Check if all required columns exist
    missing_cols = set(id_cols + [date_col] + value_cols) - set(df.columns)
    if missing_cols:
        print(f"Warning: Missing columns in {file_path}: {missing_cols}")
        # Only keep columns that exist
        id_cols = [col for col in id_cols if col in df.columns]
        value_cols = [col for col in value_cols if col in df.columns]
    
    # Only keep needed columns
    cols_to_keep = [date_col] + id_cols + value_cols
    df = df[cols_to_keep]
    
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Create a combined ID from the merging columns
    if len(id_cols) > 0:
        df['id'] = df[id_cols].astype(str).agg('_'.join, axis=1)
    else:
        # If no ID columns specified, use a constant
        df['id'] = 'value'
    
    # Check for and report duplicates
    duplicate_check = df.duplicated(subset=[date_col, 'id'], keep=False)
    if duplicate_check.any():
        dup_count = duplicate_check.sum()
        print(f"  Warning: Found {dup_count} duplicate entries with the same date and ID combination")
        
        # Display a sample of duplicates for debugging
        if dup_count > 0:
            print("  Sample of duplicates:")
            duplicates = df[duplicate_check].sort_values(by=[date_col, 'id']).head(min(5, dup_count))
            print(duplicates[[date_col, 'id'] + value_cols])
        
        # Detect numeric and non-numeric columns
        numeric_cols = []
        non_numeric_cols = []
        
        for col in value_cols:
            try:
                pd.to_numeric(df[col])
                numeric_cols.append(col)
            except (ValueError, TypeError):
                non_numeric_cols.append(col)
        
        # Report column types
        if numeric_cols:
            print(f"  Numeric columns that will be averaged: {numeric_cols}")
        if non_numeric_cols:
            print(f"  Non-numeric columns that will use 'first' value: {non_numeric_cols}")
        
        # Create an aggregation dictionary
        agg_dict = {}
        for col in numeric_cols:
            agg_dict[col] = 'mean'
        for col in non_numeric_cols:
            agg_dict[col] = 'first'
        
        # Apply the appropriate aggregation to each column
        print("  Aggregating duplicate values with mixed methods (mean for numeric, first for non-numeric)")
        df = df.groupby([date_col, 'id']).agg(agg_dict).reset_index()
    
    # Create a wide-format dataframe
    result = pd.DataFrame(index=df[date_col].unique())
    result.index = pd.to_datetime(result.index)
    result.sort_index(inplace=True)
    result.index.name = date_col
    
    # Get file type (simple name without version info)
    file_type = os.path.basename(file_path).split('_')[0]
    
    # Process each value column
    for col in value_cols:
        # Pivot to get values for each ID
        pivot = df.pivot(index=date_col, columns='id', values=col)
        
        # Rename columns with a more descriptive format: FileType_ID_Column  
        pivot.columns = [f"{id_val}_{col}" for id_val in pivot.columns]
        
        # Join with the result
        result = result.join(pivot)
    
    # Filter out columns containing "_SEM_"
    sem_columns = [col for col in result.columns if "_SEM_" in col]
    if sem_columns:
        print(f"  Removing {len(sem_columns)} columns containing '_SEM_'")
        result = result.drop(columns=sem_columns)
    
    print(f"  Processed {file_path} into {result.shape[1]} columns")
    return result

def main():
    # Main processing logic
    # input_folder_path = "../../processed_data/grouped_data/Generation"
    # output_folder_path = "../../processed_data/pivoted_data/Generation"
    input_folder_path = Path("filtered_data/Generation")
    output_folder_path = Path("pivoted_data/Generation")
    
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Process each file according to its configuration
    for file_config in config:
        file_name = file_config["Generation folder"]
        file_path = os.path.join(input_folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            result_df = process_file(file_path, file_config)
            
            if result_df is not None:
                # Generate output filename - replace the original extension with '_pivoted.csv'
                base_name = os.path.splitext(file_name)[0]
                output_file = f"{base_name}_pivoted.csv"
                output_path = os.path.join(output_folder_path, output_file)
                
                # Save individual pivoted file
                result_df.to_csv(output_path)
                
                # Print statistics for this file
                print(f"  Saved to {output_path}")
                print(f"  Shape: {result_df.shape}")
                print(f"  Date range: {result_df.index.min()} to {result_df.index.max()}")
                print(f"  Missing values: {result_df.isna().sum().sum()}/{result_df.size} " 
                     f"({100*result_df.isna().sum().sum()/result_df.size:.2f}%)")
                print(f"  Sample columns: {list(result_df.columns)[:3]}")
                print("")
        else:
            print(f"File not found: {file_path}")
    
    print("\nAll files processed and saved to individual pivoted CSVs in:")
    print(output_folder_path)

if __name__ == "__main__":
    main()

### Load

In [ ]:

import pandas as pd
import os
import json

# Configuration for each file
# Load folder

config = [
    {
        "Load folder": "ActualTotalLoad_6.1.A_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, TotalLoadValue"
    },
    {
        "Load folder": "DayAheadTotalLoadForecast_6.1.B_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, TotalLoadValue"
    },
    {
        "Load folder": "MonthAheadTotalLoadForecast_6.1.D_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, MinimumTotalLoadValue, MaximumTotalLoadValue"
    },
    {
        "Load folder": "WeekAheadTotalLoadForecast_6.1.C_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, MinimumTotalLoadValue, MaximumTotalLoadValue"
    },
    {
        "Load folder": "YearAheadTotalLoadForecast_6.1.E_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, MinimumTotalLoadValue, MaximumTotalLoadValue"
    },
    {
        "Load folder": "YearAheadForecastMargin_8.1_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, ForecastedMargin"
    }

]



def process_file(file_path, file_config):
    """
    Process a single file to convert from long format to wide format
    based on the specified configuration.
    """
    try:
        # Try reading with UTF-8
        df = pd.read_csv(file_path, low_memory=False)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try 'latin1'
            df = pd.read_csv(file_path, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            return None
    
    # Extract configuration
    date_col = file_config["Index/date column name"]
    id_cols = [col.strip() for col in file_config["Merging column"].split(",")]
    value_cols = [col.strip() for col in file_config["Keep columns"].split(",")]
    
    # Check if all required columns exist
    missing_cols = set(id_cols + [date_col] + value_cols) - set(df.columns)
    if missing_cols:
        print(f"Warning: Missing columns in {file_path}: {missing_cols}")
        # Only keep columns that exist
        id_cols = [col for col in id_cols if col in df.columns]
        value_cols = [col for col in value_cols if col in df.columns]
    
    # Only keep needed columns
    cols_to_keep = [date_col] + id_cols + value_cols
    df = df[cols_to_keep]
    
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Create a combined ID from the merging columns
    if len(id_cols) > 0:
        df['id'] = df[id_cols].astype(str).agg('_'.join, axis=1)
    else:
        # If no ID columns specified, use a constant
        df['id'] = 'value'
    
    # Check for and report duplicates
    duplicate_check = df.duplicated(subset=[date_col, 'id'], keep=False)
    if duplicate_check.any():
        dup_count = duplicate_check.sum()
        print(f"  Warning: Found {dup_count} duplicate entries with the same date and ID combination")
        
        # Display a sample of duplicates for debugging
        if dup_count > 0:
            print("  Sample of duplicates:")
            duplicates = df[duplicate_check].sort_values(by=[date_col, 'id']).head(min(5, dup_count))
            print(duplicates[[date_col, 'id'] + value_cols])
        
        # Detect numeric and non-numeric columns
        numeric_cols = []
        non_numeric_cols = []
        
        for col in value_cols:
            try:
                pd.to_numeric(df[col])
                numeric_cols.append(col)
            except (ValueError, TypeError):
                non_numeric_cols.append(col)
        
        # Report column types
        if numeric_cols:
            print(f"  Numeric columns that will be averaged: {numeric_cols}")
        if non_numeric_cols:
            print(f"  Non-numeric columns that will use 'first' value: {non_numeric_cols}")
        
        # Create an aggregation dictionary
        agg_dict = {}
        for col in numeric_cols:
            agg_dict[col] = 'mean'
        for col in non_numeric_cols:
            agg_dict[col] = 'first'
        
        # Apply the appropriate aggregation to each column
        print("  Aggregating duplicate values with mixed methods (mean for numeric, first for non-numeric)")
        df = df.groupby([date_col, 'id']).agg(agg_dict).reset_index()
    
    # Create a wide-format dataframe
    result = pd.DataFrame(index=df[date_col].unique())
    result.index = pd.to_datetime(result.index)
    result.sort_index(inplace=True)
    result.index.name = date_col
    
    # Get file type (simple name without version info)
    file_type = os.path.basename(file_path).split('_')[0]
    
    # Process each value column
    for col in value_cols:
        # Pivot to get values for each ID
        pivot = df.pivot(index=date_col, columns='id', values=col)
        
        # Rename columns with a more descriptive format: FileType_ID_Column  
        pivot.columns = [f"{id_val}_{col}" for id_val in pivot.columns]
        
        # Join with the result
        result = result.join(pivot)
    
    # Filter out columns containing "_SEM_"
    sem_columns = [col for col in result.columns if "_SEM_" in col]
    if sem_columns:
        print(f"  Removing {len(sem_columns)} columns containing '_SEM_'")
        result = result.drop(columns=sem_columns)
    
    print(f"  Processed {file_path} into {result.shape[1]} columns")
    return result

def main():
    # Main processing logic
    # input_folder_path = "../../processed_data/grouped_data/Load"
    # output_folder_path = "../../processed_data/pivoted_data/Load"
    input_folder_path = Path("filtered_data/Load")
    output_folder_path = Path("pivoted_data/Load")
    
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Process each file according to its configuration
    for file_config in config:
        file_name = file_config["Load folder"]
        file_path = os.path.join(input_folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            result_df = process_file(file_path, file_config)
            
            if result_df is not None:
                # Generate output filename - replace the original extension with '_pivoted.csv'
                base_name = os.path.splitext(file_name)[0]
                output_file = f"{base_name}_pivoted.csv"
                output_path = os.path.join(output_folder_path, output_file)
                
                # Save individual pivoted file
                result_df.to_csv(output_path)
                
                # Print statistics for this file
                print(f"  Saved to {output_path}")
                print(f"  Shape: {result_df.shape}")
                print(f"  Date range: {result_df.index.min()} to {result_df.index.max()}")
                print(f"  Missing values: {result_df.isna().sum().sum()}/{result_df.size} " 
                     f"({100*result_df.isna().sum().sum()/result_df.size:.2f}%)")
                print(f"  Sample columns: {list(result_df.columns)[:3]}")
                print("")
        else:
            print(f"File not found: {file_path}")
    
    print("\nAll files processed and saved to individual pivoted CSVs in:")
    print(output_folder_path)

if __name__ == "__main__":
    main()

### Outages

In [ ]:

import pandas as pd
import os
import json

# Configuration for each file
# Outages folder

config = [
    {
        "Outages folder": "ChangesInActualAvailabilityOfConsumptionUnits_7.1.B_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "MapCode",
        "Keep columns": "ResolutionCode, AreaTypeCode, UnavailableCapacity"
    }
        

]



def process_file(file_path, file_config):
    """
    Process a single file to convert from long format to wide format
    based on the specified configuration.
    """
    try:
        # Try reading with UTF-8
        df = pd.read_csv(file_path, low_memory=False)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try 'latin1'
            df = pd.read_csv(file_path, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            return None
    
    # Extract configuration
    date_col = file_config["Index/date column name"]
    id_cols = [col.strip() for col in file_config["Merging column"].split(",")]
    value_cols = [col.strip() for col in file_config["Keep columns"].split(",")]
    
    # Check if all required columns exist
    missing_cols = set(id_cols + [date_col] + value_cols) - set(df.columns)
    if missing_cols:
        print(f"Warning: Missing columns in {file_path}: {missing_cols}")
        # Only keep columns that exist
        id_cols = [col for col in id_cols if col in df.columns]
        value_cols = [col for col in value_cols if col in df.columns]
    
    # Only keep needed columns
    cols_to_keep = [date_col] + id_cols + value_cols
    df = df[cols_to_keep]
    
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Create a combined ID from the merging columns
    if len(id_cols) > 0:
        df['id'] = df[id_cols].astype(str).agg('_'.join, axis=1)
    else:
        # If no ID columns specified, use a constant
        df['id'] = 'value'
    
    # Check for and report duplicates
    duplicate_check = df.duplicated(subset=[date_col, 'id'], keep=False)
    if duplicate_check.any():
        dup_count = duplicate_check.sum()
        print(f"  Warning: Found {dup_count} duplicate entries with the same date and ID combination")
        
        # Display a sample of duplicates for debugging
        if dup_count > 0:
            print("  Sample of duplicates:")
            duplicates = df[duplicate_check].sort_values(by=[date_col, 'id']).head(min(5, dup_count))
            print(duplicates[[date_col, 'id'] + value_cols])
        
        # Detect numeric and non-numeric columns
        numeric_cols = []
        non_numeric_cols = []
        
        for col in value_cols:
            try:
                pd.to_numeric(df[col])
                numeric_cols.append(col)
            except (ValueError, TypeError):
                non_numeric_cols.append(col)
        
        # Report column types
        if numeric_cols:
            print(f"  Numeric columns that will be averaged: {numeric_cols}")
        if non_numeric_cols:
            print(f"  Non-numeric columns that will use 'first' value: {non_numeric_cols}")
        
        # Create an aggregation dictionary
        agg_dict = {}
        for col in numeric_cols:
            agg_dict[col] = 'mean'
        for col in non_numeric_cols:
            agg_dict[col] = 'first'
        
        # Apply the appropriate aggregation to each column
        print("  Aggregating duplicate values with mixed methods (mean for numeric, first for non-numeric)")
        df = df.groupby([date_col, 'id']).agg(agg_dict).reset_index()
    
    # Create a wide-format dataframe
    result = pd.DataFrame(index=df[date_col].unique())
    result.index = pd.to_datetime(result.index)
    result.sort_index(inplace=True)
    result.index.name = date_col
    
    # Get file type (simple name without version info)
    file_type = os.path.basename(file_path).split('_')[0]
    
    # Process each value column
    for col in value_cols:
        # Pivot to get values for each ID
        pivot = df.pivot(index=date_col, columns='id', values=col)
        
        # Rename columns with a more descriptive format: FileType_ID_Column  
        pivot.columns = [f"{id_val}_{col}" for id_val in pivot.columns]
        
        # Join with the result
        result = result.join(pivot)
    
    # Filter out columns containing "_SEM_"
    sem_columns = [col for col in result.columns if "_SEM_" in col]
    if sem_columns:
        print(f"  Removing {len(sem_columns)} columns containing '_SEM_'")
        result = result.drop(columns=sem_columns)
    
    print(f"  Processed {file_path} into {result.shape[1]} columns")
    return result

def main():
    # Main processing logic
    # input_folder_path = "../../processed_data/grouped_data/Outages"
    # output_folder_path = "../../processed_data/pivoted_data/Outages"
    
    input_folder_path = Path("filtered_data/Outages")
    output_folder_path = Path("pivoted_data/Outages")
    
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Process each file according to its configuration
    for file_config in config:
        file_name = file_config["Outages folder"]
        file_path = os.path.join(input_folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            result_df = process_file(file_path, file_config)
            
            if result_df is not None:
                # Generate output filename - replace the original extension with '_pivoted.csv'
                base_name = os.path.splitext(file_name)[0]
                output_file = f"{base_name}_pivoted.csv"
                output_path = os.path.join(output_folder_path, output_file)
                
                # Save individual pivoted file
                result_df.to_csv(output_path)
                
                # Print statistics for this file
                print(f"  Saved to {output_path}")
                print(f"  Shape: {result_df.shape}")
                print(f"  Date range: {result_df.index.min()} to {result_df.index.max()}")
                print(f"  Missing values: {result_df.isna().sum().sum()}/{result_df.size} " 
                     f"({100*result_df.isna().sum().sum()/result_df.size:.2f}%)")
                print(f"  Sample columns: {list(result_df.columns)[:3]}")
                print("")
        else:
            print(f"File not found: {file_path}")
    
    print("\nAll files processed and saved to individual pivoted CSVs in:")
    print(output_folder_path)

if __name__ == "__main__":
    main()

### Transmission

In [ ]:


import pandas as pd
import os
import json

# Configuration for each file
# Transmission folder

config = [
    {
        "Transmission folder": "ImplicitAllocationsNetPositions_12.1.E_r3_SE.csv",
        "Index/date column name": "DateTime(UTC)",
        "Merging column": "MapCode, ContractType, Direction",
        "Keep columns": "ResolutionCode, AreaTypeCode, NetPosition[MW]"
    },


]



def process_file(file_path, file_config):
    """
    Process a single file to convert from long format to wide format
    based on the specified configuration.
    """
    try:
        # Try reading with UTF-8
        df = pd.read_csv(file_path, low_memory=False)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try 'latin1'
            df = pd.read_csv(file_path, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            return None
    
    # Extract configuration
    date_col = file_config["Index/date column name"]
    id_cols = [col.strip() for col in file_config["Merging column"].split(",")]
    value_cols = [col.strip() for col in file_config["Keep columns"].split(",")]
    
    # Check if all required columns exist
    missing_cols = set(id_cols + [date_col] + value_cols) - set(df.columns)
    if missing_cols:
        print(f"Warning: Missing columns in {file_path}: {missing_cols}")
        # Only keep columns that exist
        id_cols = [col for col in id_cols if col in df.columns]
        value_cols = [col for col in value_cols if col in df.columns]
    
    # Only keep needed columns
    cols_to_keep = [date_col] + id_cols + value_cols
    df = df[cols_to_keep]
    
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Create a combined ID from the merging columns
    if len(id_cols) > 0:
        df['id'] = df[id_cols].astype(str).agg('_'.join, axis=1)
    else:
        # If no ID columns specified, use a constant
        df['id'] = 'value'
    
    # Check for and report duplicates
    duplicate_check = df.duplicated(subset=[date_col, 'id'], keep=False)
    if duplicate_check.any():
        dup_count = duplicate_check.sum()
        print(f"  Warning: Found {dup_count} duplicate entries with the same date and ID combination")
        
        # Display a sample of duplicates for debugging
        if dup_count > 0:
            print("  Sample of duplicates:")
            duplicates = df[duplicate_check].sort_values(by=[date_col, 'id']).head(min(5, dup_count))
            print(duplicates[[date_col, 'id'] + value_cols])
        
        # Detect numeric and non-numeric columns
        numeric_cols = []
        non_numeric_cols = []
        
        for col in value_cols:
            try:
                pd.to_numeric(df[col])
                numeric_cols.append(col)
            except (ValueError, TypeError):
                non_numeric_cols.append(col)
        
        # Report column types
        if numeric_cols:
            print(f"  Numeric columns that will be averaged: {numeric_cols}")
        if non_numeric_cols:
            print(f"  Non-numeric columns that will use 'first' value: {non_numeric_cols}")
        
        # Create an aggregation dictionary
        agg_dict = {}
        for col in numeric_cols:
            agg_dict[col] = 'mean'
        for col in non_numeric_cols:
            agg_dict[col] = 'first'
        
        # Apply the appropriate aggregation to each column
        print("  Aggregating duplicate values with mixed methods (mean for numeric, first for non-numeric)")
        df = df.groupby([date_col, 'id']).agg(agg_dict).reset_index()
    
    # Create a wide-format dataframe
    result = pd.DataFrame(index=df[date_col].unique())
    result.index = pd.to_datetime(result.index)
    result.sort_index(inplace=True)
    result.index.name = date_col
    
    # Get file type (simple name without version info)
    file_type = os.path.basename(file_path).split('_')[0]
    
    # Process each value column
    for col in value_cols:
        # Pivot to get values for each ID
        pivot = df.pivot(index=date_col, columns='id', values=col)
        
        # Rename columns with a more descriptive format: FileType_ID_Column  
        pivot.columns = [f"{id_val}_{col}" for id_val in pivot.columns]
        
        # Join with the result
        result = result.join(pivot)
    
    # Filter out columns containing "_SEM_"
    sem_columns = [col for col in result.columns if "_SEM_" in col]
    if sem_columns:
        print(f"  Removing {len(sem_columns)} columns containing '_SEM_'")
        result = result.drop(columns=sem_columns)
    
    print(f"  Processed {file_path} into {result.shape[1]} columns")
    return result

def main():
    # Main processing logic
    input_folder_path = Path("filtered_data/Transmission")
    output_folder_path = Path("pivoted_data/Transmission")
    
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Process each file according to its configuration
    for file_config in config:
        file_name = file_config["Transmission folder"]
        file_path = os.path.join(input_folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            result_df = process_file(file_path, file_config)
            
            if result_df is not None:
                # Generate output filename - replace the original extension with '_pivoted.csv'
                base_name = os.path.splitext(file_name)[0]
                output_file = f"{base_name}_pivoted.csv"
                output_path = os.path.join(output_folder_path, output_file)
                
                # Save individual pivoted file
                result_df.to_csv(output_path)
                
                # Print statistics for this file
                print(f"  Saved to {output_path}")
                print(f"  Shape: {result_df.shape}")
                print(f"  Date range: {result_df.index.min()} to {result_df.index.max()}")
                print(f"  Missing values: {result_df.isna().sum().sum()}/{result_df.size} " 
                     f"({100*result_df.isna().sum().sum()/result_df.size:.2f}%)")
                print(f"  Sample columns: {list(result_df.columns)[:3]}")
                print("")
        else:
            print(f"File not found: {file_path}")
    
    print("\nAll files processed and saved to individual pivoted CSVs in:")
    print(output_folder_path)

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import os

# Configuration for the file
config = [

    {
        "Transmission folder": "FlowBasedAllocationsCongestionincomeDaily_12.1.E_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, Revenue"
    },
    {
        "Transmission folder": "ForecastedMonthAheadTransferCapacities_11.1_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, ForecastTransferCapacity"
    },
    {
        "Transmission folder": "ForecastedWeekAheadTransferCapacities_11.1_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, ForecastTransferCapacity"
    },
    {
        "Transmission folder": "ForecastedYearAheadTransferCapacities_11.1_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, ForecastTransferCapacity"
    },
    {
        "Transmission folder": "ImplicitAllocationsCongestionIncomeDaily_12.1.E_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, Revenue"
    },
    {
        "Transmission folder": "OfferedDayAheadTransferCapacitylmplicit_11.1_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, Capacity"
    },
    {
        "Transmission folder": "OfferedIntradayTransferCapacityImplicit_11.1_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, Capacity"
    },
    {
        "Transmission folder": "OfferedTransferCapacitiesContinuous_11.1_r3_SE.csv",
        "Index/date column name": "MTU(UTC)",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, ContractType, Capacity[MW]"
    },
    {
        "Transmission folder": "OfferedTransferCapacitiesContinuousEvolution_11.1_r3_SE.csv",
        "Index/date column name": "MTU(UTC)",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, ContractType, Capacity[MW]"
    },
    {
        "Transmission folder": "PhysicalFlows_12.1.G_SE.csv",
        "Index/date column name": "DateTime",
        "Merging column": "OutMapCode",
        "Keep columns": "ResolutionCode, FlowValue"
    }

]

def process_file(file_path, file_config):
    """
    Process a single file to convert from long format to wide format
    based on the specified configuration with special handling for SE regions.
    """
    try:
        # Try reading with UTF-8
        df = pd.read_csv(file_path, low_memory=False)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try 'latin1'
            df = pd.read_csv(file_path, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            return None
    
    # Extract configuration
    date_col = file_config["Index/date column name"]
    value_cols = [col.strip() for col in file_config["Keep columns"].split(",")]
    
    # Check if required columns exist
    required_cols = [date_col, "OutMapCode", "InMapCode"] + value_cols
    missing_cols = set(required_cols) - set(df.columns)
    if missing_cols:
        print(f"Warning: Missing columns in {file_path}: {missing_cols}")
        return None
    
    # Only keep needed columns
    cols_to_keep = [date_col, "OutMapCode", "InMapCode"] + value_cols
    df = df[cols_to_keep]
    
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Define Swedish bidding zones
    swedish_zones = ['SE', 'SE1', 'SE2', 'SE3', 'SE4']
    
    # Filter for rows where either OutMapCode or InMapCode is a Swedish zone
    df_se = df[(df['OutMapCode'].isin(swedish_zones)) | (df['InMapCode'].isin(swedish_zones))]
    
    # Create a direction-based identifier
    df_se['flow_id'] = df_se.apply(
        lambda row: f"FROM_{row['OutMapCode']}_TO_{row['InMapCode']}", 
        axis=1
    )
    
    # Check for duplicates with the new identifier
    duplicate_check = df_se.duplicated(subset=[date_col, 'flow_id'], keep=False)
    if duplicate_check.any():
        dup_count = duplicate_check.sum()
        print(f"  Warning: Found {dup_count} duplicate entries with the same date and flow ID")
        
        # Detect numeric and non-numeric columns
        numeric_cols = []
        non_numeric_cols = []
        
        for col in value_cols:
            try:
                pd.to_numeric(df_se[col])
                numeric_cols.append(col)
            except (ValueError, TypeError):
                non_numeric_cols.append(col)
        
        # Create an aggregation dictionary
        agg_dict = {}
        for col in numeric_cols:
            agg_dict[col] = 'mean'
        for col in non_numeric_cols:
            agg_dict[col] = 'first'
        
        # Apply the appropriate aggregation to each column
        print("  Aggregating duplicate values with mixed methods (mean for numeric, first for non-numeric)")
        df_se = df_se.groupby([date_col, 'flow_id']).agg(agg_dict).reset_index()
    
    # Create a wide-format dataframe
    result = pd.DataFrame(index=df_se[date_col].unique())
    result.index = pd.to_datetime(result.index)
    result.sort_index(inplace=True)
    result.index.name = date_col
    
    # Process each value column
    for col in value_cols:
        # Pivot to get values for each flow ID
        pivot = df_se.pivot(index=date_col, columns='flow_id', values=col)
        
        # Rename columns with a more descriptive format
        pivot.columns = [f"{flow_id}_{col}" for flow_id in pivot.columns]
        
        # Join with the result
        result = result.join(pivot)
    
    print(f"  Processed {file_path} into {result.shape[1]} columns")
    return result

def main():
    # Main processing logic
    input_folder_path = Path("filtered_data/Transmission")
    output_folder_path = Path("pivoted_data/Transmission")
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Process each file according to its configuration
    for file_config in config:
        file_name = file_config["Transmission folder"]
        file_path = os.path.join(input_folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            result_df = process_file(file_path, file_config)
            
            if result_df is not None:
                # Generate output filename - replace the original extension with '_pivoted.csv'
                base_name = os.path.splitext(file_name)[0]
                output_file = f"{base_name}_pivoted.csv"
                output_path = os.path.join(output_folder_path, output_file)
                
                # Save individual pivoted file
                result_df.to_csv(output_path)
                
                # Print statistics for this file
                print(f"  Saved to {output_path}")
                print(f"  Shape: {result_df.shape}")
                print(f"  Date range: {result_df.index.min()} to {result_df.index.max()}")
                print(f"  Missing values: {result_df.isna().sum().sum()}/{result_df.size} " 
                     f"({100*result_df.isna().sum().sum()/result_df.size:.2f}%)")
                print(f"  Sample columns: {list(result_df.columns)[:3]}")
                print("")
        else:
            print(f"File not found: {file_path}")
    
    print("\nAll files processed and saved to individual pivoted CSVs in:")
    print(output_folder_path)

if __name__ == "__main__":
    main()

## Clean up column names

In [40]:
import pandas as pd
from pathlib import Path
import os

# Define the source and target directories
source_dir = Path("pivoted_data")
target_dir = Path("pivoted_data_filtered")

# Dictionary for replacements
replacements = {
    "Automatic Frequency Restoration Reserve (aFRR)": "aFRR",
    "Manual Frequency Restoration Reserve (mFRR)": "mFRR",
    "Frequency Containment Reserve (FCR)": "FCR"
}

def rename_columns(df):
    """Rename columns if they contain specific strings"""
    new_columns = {}
    modified = False
    
    for col in df.columns:
        new_col = col
        for old, new in replacements.items():
            if old in col:
                new_col = col.replace(old, new)
                modified = True
        if new_col != col:
            new_columns[col] = new_col
    
    if modified:
        df = df.rename(columns=new_columns)
    return df, modified

# Process all CSV files in subdirectories
for source_path in source_dir.rglob("*.csv"):
    # Read the CSV file with low_memory=False to avoid dtype warnings
    df = pd.read_csv(source_path, low_memory=False)
    
    # Rename columns and check if any modifications were made
    df, was_modified = rename_columns(df)
    
    # Only save if modifications were made
    if was_modified:
        # Create corresponding directory structure in target_dir
        relative_path = source_path.relative_to(source_dir)
        target_path = target_dir / relative_path
        
        # Create directory if it doesn't exist
        target_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the modified CSV
        df.to_csv(target_path, index=False)
        print(f"Processed and saved: {relative_path}")

### Sanity check column names

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

# Define the source and target directories
source_dir = Path("pivoted_data")
target_dir = Path("pivoted_data_filtered")

# Dictionary for replacements
replacements = {
    "Automatic Frequency Restoration Reserve (aFRR)": "aFRR",
    "Manual Frequency Restoration Reserve (mFRR)": "mFRR",
    "Frequency Containment Reserve (FCR)": "FCR"
}

def get_modified_column_name(original_col):
    """Convert original column name to its modified version"""
    modified_col = original_col
    for old, new in replacements.items():
        if old in original_col:
            modified_col = original_col.replace(old, new)
    return modified_col

def compare_dataframes(original_df, modified_df, file_path):
    """Compare two dataframes and report any differences"""
    
    # Check if the data shape is the same
    if original_df.shape != modified_df.shape:
        print(f"\n {file_path}: Shape mismatch!")
        print(f"Original shape: {original_df.shape}")
        print(f"Modified shape: {modified_df.shape}")
        return False
    
    # Get expected column names after modification
    expected_cols = [get_modified_column_name(col) for col in original_df.columns]
    actual_cols = modified_df.columns.tolist()
    
    # Check if columns match
    if set(expected_cols) != set(actual_cols):
        print(f"\n {file_path}: Column names don't match as expected!")
        print("Mismatched columns:")
        print("Expected but not found:", set(expected_cols) - set(actual_cols))
        print("Found but not expected:", set(actual_cols) - set(expected_cols))
        return False
    
    # Reorder modified_df columns to match the expected order
    modified_df = modified_df[expected_cols]
    
    # Compare data values column by column
    for col in original_df.columns:
        modified_col = get_modified_column_name(col)
        
        # Convert to string to handle mixed types
        orig_series = original_df[col].astype(str)
        mod_series = modified_df[modified_col].astype(str)
        
        if not orig_series.equals(mod_series):
            print(f"\n {file_path}: Data values are different in column {col}!")
            return False
    
    print(f"\n {file_path}: Data is identical (only column names changed)")
    return True

# Find all modified files in target directory
print("Starting comparison...")
for target_path in target_dir.rglob("*.csv"):
    # Get corresponding source file path
    relative_path = target_path.relative_to(target_dir)
    source_path = source_dir / relative_path
    
    if source_path.exists():
        print(f"\nComparing {relative_path}")
        # Read both files
        original_df = pd.read_csv(source_path, low_memory=False)
        modified_df = pd.read_csv(target_path, low_memory=False)
        
        # Compare the files
        compare_dataframes(original_df, modified_df, relative_path)

## Clean up columns

In [ ]:
import pandas as pd
from pathlib import Path
import shutil

# Define source and target directories
source_dir = Path('pivoted_data')

target_dir = Path('pivoted_data_short')

# Create target directory if it doesn't exist
target_dir.mkdir(parents=True, exist_ok=True)

# Columns to check for
columns_to_check = ['ResolutionCode', 'AreaTypeCode', 'OutAreaTypeCode', 'InAreaTypeCode', 'Currency', 'TimeHorizon', 'PriceType', 'ReserveSource', 'ContractType', 'SourceName', 'TypeOfProduct', 'Replacement Reserve (RR)', 'Resolution', 'Action']

# Loop through all subdirectories in source directory
for subdir in source_dir.iterdir():
    if subdir.is_dir():
        # Create corresponding subdirectory in target
        new_subdir = target_dir / subdir.name
        new_subdir.mkdir(parents=True, exist_ok=True)
        
        # Process all CSV files in the subdirectory
        for csv_file in subdir.glob('*.csv'):
            # Read the CSV
            df = pd.read_csv(csv_file, low_memory=False)
            # print(df.head())
            
            # Check if any column contains the specified strings
            columns_to_drop = [col for col in df.columns 
                             if any(check in col for check in columns_to_check)]
            
            # Drop the columns if found
            if columns_to_drop:
                df = df.drop(columns=columns_to_drop)
                
            # Save to new location
            new_file_path = new_subdir / csv_file.name
            df.to_csv(new_file_path, index=False)
            
            print(f"Processed {csv_file.name}: Dropped {len(columns_to_drop)} columns")

## Encode data and remove deprecated data

### Balancing folder

In [ ]:
import os
import pandas as pd

# Define directories
input_dir = Path("pivoted_data_short/Balancing")
output_dir = Path("pivoted_data_encoded/Balancing")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all CSV files in the Balancing folder
for file in os.listdir(input_dir):
    if file.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(input_dir, file)
        
        # Create new filename by keeping only part before first underscore
        base_name = file.split('_')[0] + '.csv'
        output_file_path = os.path.join(output_dir, base_name)

        print(f"Processing: {file}")
        
        # Read CSV
        df = pd.read_csv(file_path, low_memory=False)  # Added low_memory=False to avoid DtypeWarning
        
        # Identify first column as DateTime (if applicable)
        df.rename(columns={df.columns[0]: "DateTime"}, inplace=True)
        df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")

        # Identify categorical columns (excluding DateTime)
        categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
        if "DateTime" in categorical_columns:
            categorical_columns.remove("DateTime")

        # Identify numerical (continuous) columns
        # numerical_columns = df.select_dtypes(include=["number"]).columns.tolist()

        # One-hot encoding categorical variables while avoiding encoding missing values
        for col in categorical_columns:
            dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False, dtype=int)  # Avoid encoding NaN
            df = pd.concat([df, dummies], axis=1)
            df.drop(columns=[col], inplace=True)

        # Standardize numerical (continuous) columns while preserving NaNs
        # if numerical_columns:
        #     scaler = StandardScaler()
        #     df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

        # Save the processed file
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

print("All Balancing files processed successfully!") 

In [ ]:
import pandas as pd
import os
from pathlib import Path

folder_path = Path("pivoted_data_encoded/Balancing")


# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Create variable name from filename
        df_name = filename.replace('.csv', '')
        
        # Read the CSV file
        df = pd.read_csv(file_path, low_memory=False)
        
        # Convert DateTime column to datetime
        df['DateTime'] = pd.to_datetime(df['DateTime'])
        
        # Get list of columns excluding DateTime
        columns_to_check = [col for col in df.columns if col != 'DateTime']
        columns_to_keep = ['DateTime']  # Always keep DateTime column
        
        # Check each column
        for col in columns_to_check:
            # Check if column has any data after 2023
            has_recent_data = df[df['DateTime'].dt.year >= 2023][col].notna().any()
            
            # Calculate missing percentage for pre-2023 data
            pre_2023_data = df[df['DateTime'].dt.year < 2023][col]
            missing_percentage = (pre_2023_data.isna().sum() / len(pre_2023_data)) * 100
            
            # Keep column if it has recent data OR if pre-2023 missing data is less than threshold
            if has_recent_data or missing_percentage < 95.45:
                columns_to_keep.append(col)
        
        # Filter DataFrame to keep only selected columns
        df_filtered = df[columns_to_keep]
        
        # Print summary
        print(f"\nFile: {filename}")
        print(f"Original columns: {len(df.columns)}")
        print(f"Columns after filtering: {len(df_filtered.columns)}")
        print(f"Removed {len(df.columns) - len(df_filtered.columns)} columns")
        
        # Save filtered DataFrame back to CSV
        output_path = os.path.join(folder_path, filename)
        df_filtered.to_csv(output_path, index=False)
        print(f"Saved filtered data to: {output_path}")

print("\nAll files processed successfully!")

### Congestion Management

In [ ]:
import os
import pandas as pd

# Define directories
input_dir = Path("pivoted_data_short/Congestion Management")
output_dir = Path("pivoted_data_encoded/Congestion Management")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all CSV files in the Congestion Management folder
for file in os.listdir(input_dir):
    if file.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(input_dir, file)
        
        # Create new filename by keeping only part before first underscore
        base_name = file.split('_')[0] + '.csv'
        output_file_path = os.path.join(output_dir, base_name)

        print(f"Processing: {file}")
        
        # Read CSV
        df = pd.read_csv(file_path, low_memory=False)  # Added low_memory=False to avoid DtypeWarning
        
        # Identify first column as DateTime (if applicable)
        df.rename(columns={df.columns[0]: "DateTime"}, inplace=True)
        df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")

        # Identify categorical columns (excluding DateTime)
        categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
        if "DateTime" in categorical_columns:
            categorical_columns.remove("DateTime")

        # Identify numerical (continuous) columns
        # numerical_columns = df.select_dtypes(include=["number"]).columns.tolist()

        # One-hot encoding categorical variables while avoiding encoding missing values
        for col in categorical_columns:
            dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False, dtype=int)  # Avoid encoding NaN
            df = pd.concat([df, dummies], axis=1)
            df.drop(columns=[col], inplace=True)

        # Standardize numerical (continuous) columns while preserving NaNs
        # if numerical_columns:
        #     scaler = StandardScaler()
        #     df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

        # Save the processed file
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

print("All Congestion Management files processed successfully!") 

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Using relative path from current working directory
folder_path = Path("pivoted_data_encoded/Congestion Management")

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Create variable name from filename
        df_name = filename.replace('.csv', '')
        
        # Read the CSV file
        df = pd.read_csv(file_path, low_memory=False)
        
        # Convert DateTime column to datetime
        df['DateTime'] = pd.to_datetime(df['DateTime'])
        
        # Get list of columns excluding DateTime
        columns_to_check = [col for col in df.columns if col != 'DateTime']
        columns_to_keep = ['DateTime']  # Always keep DateTime column
        
        # Check each column
        for col in columns_to_check:
            # Check if column has any data after 2023
            has_recent_data = df[df['DateTime'].dt.year >= 2023][col].notna().any()
            
            # Calculate missing percentage for pre-2023 data
            pre_2023_data = df[df['DateTime'].dt.year < 2023][col]
            missing_percentage = (pre_2023_data.isna().sum() / len(pre_2023_data)) * 100
            
            # Keep column if it has recent data OR if pre-2023 missing data is less than threshold
            if has_recent_data or missing_percentage < 95.45:
                columns_to_keep.append(col)
        
        # Filter DataFrame to keep only selected columns
        df_filtered = df[columns_to_keep]
        
        # Print summary
        print(f"\nFile: {filename}")
        print(f"Original columns: {len(df.columns)}")
        print(f"Columns after filtering: {len(df_filtered.columns)}")
        print(f"Removed {len(df.columns) - len(df_filtered.columns)} columns")
        
        # Save filtered DataFrame back to CSV
        output_path = os.path.join(folder_path, filename)
        df_filtered.to_csv(output_path, index=False)
        print(f"Saved filtered data to: {output_path}")

print("\nAll files processed successfully!")

### Generation folder

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define directories
input_dir = Path("pivoted_data_short/Generation")
output_dir = Path("pivoted_data_encoded/Generation")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all CSV files in the Generation folder
for file in os.listdir(input_dir):
    if file.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(input_dir, file)
        
        # Create new filename by keeping only part before first underscore
        base_name = file.split('_')[0] + '.csv'
        output_file_path = os.path.join(output_dir, base_name)

        print(f"Processing: {file}")
        
        # Read CSV
        df = pd.read_csv(file_path, low_memory=False)  # Added low_memory=False to avoid DtypeWarning
        
        # Identify first column as DateTime (if applicable)
        df.rename(columns={df.columns[0]: "DateTime"}, inplace=True)
        df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")

        # Identify categorical columns (excluding DateTime)
        categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
        if "DateTime" in categorical_columns:
            categorical_columns.remove("DateTime")



        # One-hot encoding categorical variables while avoiding encoding missing values
        for col in categorical_columns:
            dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False, dtype=int)  # Avoid encoding NaN, dtype = 1 or 0
            df = pd.concat([df, dummies], axis=1)
            df.drop(columns=[col], inplace=True)


        # Save the processed file
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

print("All Generation files processed successfully!") 

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Using relative path from current working directory
folder_path = Path("pivoted_data_encoded/Generation")

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Create variable name from filename
        df_name = filename.replace('.csv', '')
        
        # Read the CSV file
        df = pd.read_csv(file_path, low_memory=False)
        
        # Convert DateTime column to datetime
        df['DateTime'] = pd.to_datetime(df['DateTime'])
        
        # Get list of columns excluding DateTime
        columns_to_check = [col for col in df.columns if col != 'DateTime']
        columns_to_keep = ['DateTime']  # Always keep DateTime column
        
        # Check each column
        for col in columns_to_check:
            # Check if column has any data after 2023
            has_recent_data = df[df['DateTime'].dt.year >= 2023][col].notna().any()
            
            # Calculate missing percentage for pre-2023 data
            pre_2023_data = df[df['DateTime'].dt.year < 2023][col]
            missing_percentage = (pre_2023_data.isna().sum() / len(pre_2023_data)) * 100
            
            # Keep column if it has recent data OR if pre-2023 missing data is less than threshold
            if has_recent_data or missing_percentage < 95.45:
                columns_to_keep.append(col)
        
        # Filter DataFrame to keep only selected columns
        df_filtered = df[columns_to_keep]
        
        # Print summary
        print(f"\nFile: {filename}")
        print(f"Original columns: {len(df.columns)}")
        print(f"Columns after filtering: {len(df_filtered.columns)}")
        print(f"Removed {len(df.columns) - len(df_filtered.columns)} columns")
        
        # Save filtered DataFrame back to CSV
        output_path = os.path.join(folder_path, filename)
        df_filtered.to_csv(output_path, index=False)
        print(f"Saved filtered data to: {output_path}")

print("\nAll files processed successfully!")

### Load folder

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define directories
input_dir = Path("pivoted_data_short/Load")
output_dir = Path("pivoted_data_encoded/Load")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all CSV files in the Load folder
for file in os.listdir(input_dir):
    if file.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(input_dir, file)
        
        # Create new filename by keeping only part before first underscore
        base_name = file.split('_')[0] + '.csv'
        output_file_path = os.path.join(output_dir, base_name)

        print(f"Processing: {file}")
        
        # Read CSV
        df = pd.read_csv(file_path, low_memory=False)  # Added low_memory=False to avoid DtypeWarning
        
        # Identify first column as DateTime (if applicable)
        df.rename(columns={df.columns[0]: "DateTime"}, inplace=True)
        df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")

        # Identify categorical columns (excluding DateTime)
        categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
        if "DateTime" in categorical_columns:
            categorical_columns.remove("DateTime")

        # One-hot encoding categorical variables while avoiding encoding missing values
        for col in categorical_columns:
            dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False, dtype=int)  # Avoid encoding NaN, dtype = 1 or 0
            df = pd.concat([df, dummies], axis=1)
            df.drop(columns=[col], inplace=True)

        # Save the processed file
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

print("All Load files processed successfully!") 

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Using relative path from current working directory
folder_path = Path("pivoted_data_encoded/Load")

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Create variable name from filename
        df_name = filename.replace('.csv', '')
        
        # Read the CSV file
        df = pd.read_csv(file_path, low_memory=False)
        
        # Convert DateTime column to datetime
        df['DateTime'] = pd.to_datetime(df['DateTime'])
        
        # Get list of columns excluding DateTime
        columns_to_check = [col for col in df.columns if col != 'DateTime']
        columns_to_keep = ['DateTime']  # Always keep DateTime column
        
        # Check each column
        for col in columns_to_check:
            # Check if column has any data after 2023
            has_recent_data = df[df['DateTime'].dt.year >= 2023][col].notna().any()
            
            # Calculate missing percentage for pre-2023 data
            pre_2023_data = df[df['DateTime'].dt.year < 2023][col]
            missing_percentage = (pre_2023_data.isna().sum() / len(pre_2023_data)) * 100
            
            # Keep column if it has recent data OR if pre-2023 missing data is less than threshold
            if has_recent_data or missing_percentage < 95.45:
                columns_to_keep.append(col)
        
        # Filter DataFrame to keep only selected columns
        df_filtered = df[columns_to_keep]
        
        # Print summary
        print(f"\nFile: {filename}")
        print(f"Original columns: {len(df.columns)}")
        print(f"Columns after filtering: {len(df_filtered.columns)}")
        print(f"Removed {len(df.columns) - len(df_filtered.columns)} columns")
        
        # Save filtered DataFrame back to CSV
        output_path = os.path.join(folder_path, filename)
        df_filtered.to_csv(output_path, index=False)
        print(f"Saved filtered data to: {output_path}")

print("\nAll files processed successfully!")

### Outages folder

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define directories
input_dir = Path("pivoted_data_short/Outages")
output_dir = Path("pivoted_data_encoded/Outages")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all CSV files in the Outages folder
for file in os.listdir(input_dir):
    if file.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(input_dir, file)
        
        # Create new filename by keeping only part before first underscore
        base_name = file.split('_')[0] + '.csv'
        output_file_path = os.path.join(output_dir, base_name)

        print(f"Processing: {file}")
        
        # Read CSV
        df = pd.read_csv(file_path, low_memory=False)  # Added low_memory=False to avoid DtypeWarning
        
        # Remove columns containing "_Type"
        type_columns = [col for col in df.columns if "_Type" in col]
        if type_columns:
            df = df.drop(columns=type_columns)
            print(f"Removed {len(type_columns)} Type columns: {type_columns}")
        
        # Identify first column as DateTime (if applicable)
        df.rename(columns={df.columns[0]: "DateTime"}, inplace=True)
        df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")

        # Identify categorical columns (excluding DateTime)
        categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
        if "DateTime" in categorical_columns:
            categorical_columns.remove("DateTime")

        # One-hot encoding categorical variables while avoiding encoding missing values
        for col in categorical_columns:
            dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False, dtype=int)  # Avoid encoding NaN, dtype = 1 or 0
            df = pd.concat([df, dummies], axis=1)
            df.drop(columns=[col], inplace=True)

        # Save the processed file
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

print("All Outages files processed successfully!")

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Using relative path from current working directory
folder_path = Path("pivoted_data_encoded/Outages")

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Create variable name from filename
        df_name = filename.replace('.csv', '')
        
        # Read the CSV file
        df = pd.read_csv(file_path, low_memory=False)
        
        # Convert DateTime column to datetime
        df['DateTime'] = pd.to_datetime(df['DateTime'])
        
        # Get list of columns excluding DateTime
        columns_to_check = [col for col in df.columns if col != 'DateTime']
        columns_to_keep = ['DateTime']  # Always keep DateTime column
        
        # Check each column
        for col in columns_to_check:
            # Check if column has any data after 2023
            has_recent_data = df[df['DateTime'].dt.year >= 2023][col].notna().any()
            
            # Calculate missing percentage for pre-2023 data
            pre_2023_data = df[df['DateTime'].dt.year < 2023][col]
            missing_percentage = (pre_2023_data.isna().sum() / len(pre_2023_data)) * 100
            
            # Keep column if it has recent data OR if pre-2023 missing data is less than threshold
            if has_recent_data or missing_percentage < 95.45:
                columns_to_keep.append(col)
        
        # Filter DataFrame to keep only selected columns
        df_filtered = df[columns_to_keep]
        
        # Print summary
        print(f"\nFile: {filename}")
        print(f"Original columns: {len(df.columns)}")
        print(f"Columns after filtering: {len(df_filtered.columns)}")
        print(f"Removed {len(df.columns) - len(df_filtered.columns)} columns")
        
        # Save filtered DataFrame back to CSV
        output_path = os.path.join(folder_path, filename)
        df_filtered.to_csv(output_path, index=False)
        print(f"Saved filtered data to: {output_path}")

print("\nAll files processed successfully!")

### Transmission folder

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define directories
input_dir = Path("pivoted_data_short/Transmission")
output_dir = Path("pivoted_data_encoded/Transmission")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all CSV files in the Transmission folder
for file in os.listdir(input_dir):
    if file.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(input_dir, file)
        
        # Create new filename by keeping only part before first underscore
        base_name = file.split('_')[0] + '.csv'
        output_file_path = os.path.join(output_dir, base_name)

        print(f"Processing: {file}")
        
        # Read CSV
        df = pd.read_csv(file_path, low_memory=False)  # Added low_memory=False to avoid DtypeWarning
        
        # Identify first column as DateTime (if applicable)
        df.rename(columns={df.columns[0]: "DateTime"}, inplace=True)
        df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")

        # Identify categorical columns (excluding DateTime)
        categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
        if "DateTime" in categorical_columns:
            categorical_columns.remove("DateTime")



        # One-hot encoding categorical variables while avoiding encoding missing values
        for col in categorical_columns:
            dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False, dtype=int)  # Avoid encoding NaN, dtype = 1 or 0
            df = pd.concat([df, dummies], axis=1)
            df.drop(columns=[col], inplace=True)


        # Save the processed file
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

print("All Transmission files processed successfully!") 

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Using relative path from current working directory
current_dir = os.getcwd()
folder_path = Path("pivoted_data_encoded/Transmission")

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Create variable name from filename
        df_name = filename.replace('.csv', '')
        
        # Read the CSV file
        df = pd.read_csv(file_path, low_memory=False)
        
        # Convert DateTime column to datetime
        df['DateTime'] = pd.to_datetime(df['DateTime'])
        
        # Get list of columns excluding DateTime
        columns_to_check = [col for col in df.columns if col != 'DateTime']
        columns_to_keep = ['DateTime']  # Always keep DateTime column
        
        # Check each column
        for col in columns_to_check:
            # Check if column has any data after 2023
            has_recent_data = df[df['DateTime'].dt.year >= 2023][col].notna().any()
            
            # Calculate missing percentage for pre-2023 data
            pre_2023_data = df[df['DateTime'].dt.year < 2023][col]
            missing_percentage = (pre_2023_data.isna().sum() / len(pre_2023_data)) * 100
            
            # Keep column if it has recent data OR if pre-2023 missing data is less than threshold
            if has_recent_data or missing_percentage < 95.45:
                columns_to_keep.append(col)
        
        # Filter DataFrame to keep only selected columns
        df_filtered = df[columns_to_keep]
        
        # Print summary
        print(f"\nFile: {filename}")
        print(f"Original columns: {len(df.columns)}")
        print(f"Columns after filtering: {len(df_filtered.columns)}")
        print(f"Removed {len(df.columns) - len(df_filtered.columns)} columns")
        
        # Save filtered DataFrame back to CSV
        output_path = os.path.join(folder_path, filename)
        df_filtered.to_csv(output_path, index=False)
        print(f"Saved filtered data to: {output_path}")

print("\nAll files processed successfully!")

## Combine into Master Data csv

In [ ]:
import os
import pandas as pd
from pathlib import Path

def combine_csv_files():
    # Define the base directory and output path
    base_dir = Path("pivoted_data_encoded")
    output_path = Path("master_data/resultsset.csv")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    print("Starting CSV combination process...")
    
    # Initialize an empty DataFrame to store the combined data
    master_df = None
    
    # Walk through all directories
    for root, dirs, files in os.walk(base_dir):
        # Skip the base directory itself
        if root == base_dir:
            continue
            
        category = os.path.basename(root)  # Get the category name (folder name)
        print(f"\nProcessing {category} folder...")
        
        # Process each CSV file in the current directory
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"  Reading {file}...")
                
                try:
                    # Read the CSV file
                    df = pd.read_csv(file_path, low_memory=False)
                    
                    # Convert DateTime to datetime type
                    df['DateTime'] = pd.to_datetime(df['DateTime'])
                    
                    # Filter data up to 2025-01-31
                    # df = df[df['DateTime'] <= '2025-01-31']
                    
                    # Check if data has half-hourly frequency
                    time_diffs = df['DateTime'].diff().dropna()
                    if len(time_diffs) > 0 and time_diffs.min() < pd.Timedelta(hours=1):
                        print(f"    Detected sub-hourly data in {file}, aggregating to hourly...")
                        
                        # Round DateTime to nearest hour
                        df['HourlyDateTime'] = df['DateTime'].dt.floor('h')
                        
                        # Group by hour and aggregate
                        # For numeric columns, use mean
                        # For categorical/object columns, use first value
                        numeric_cols = df.select_dtypes(include=['number']).columns
                        object_cols = df.select_dtypes(include=['object']).columns.difference(['DateTime'])
                        
                        agg_dict = {}
                        for col in df.columns:
                            if col in numeric_cols:
                                agg_dict[col] = 'sum'
                            elif col in object_cols:
                                agg_dict[col] = 'first'
                        
                        # Remove DateTime from aggregation
                        if 'DateTime' in agg_dict:
                            del agg_dict['DateTime']
                        if 'HourlyDateTime' in agg_dict:
                            del agg_dict['HourlyDateTime']
                        
                        # Perform aggregation
                        df = df.groupby('HourlyDateTime').agg(agg_dict).reset_index()
                        df.rename(columns={'HourlyDateTime': 'DateTime'}, inplace=True)
                    
                    # Set DateTime as index
                    df.set_index('DateTime', inplace=True)
                    
                    # Add category prefix to column names (except DateTime)
                    df.columns = [f"{category}_{file.replace('.csv', '')}_{col}" for col in df.columns]
                    
                    # Merge with master_df
                    if master_df is None:
                        master_df = df
                    else:
                        master_df = master_df.join(df, how='outer')
                    
                    print(f"    Added {len(df.columns)} columns from {file}")
                    
                except Exception as e:
                    print(f"    Error processing {file}: {str(e)}")
                    continue
    
    if master_df is not None:
        # Sort index to ensure chronological order
        master_df.sort_index(inplace=True)
        
        # Apply filtering to remove columns with no recent data or mostly missing historical data
        print("\nFiltering columns based on data availability...")
        
        original_columns = len(master_df.columns)
        columns_to_keep = []
        
        for col in master_df.columns:
            # Check if column has any data after 2023
            has_recent_data = master_df.loc[master_df.index >= '2023-01-01', col].notna().any()
            
            # Calculate missing percentage for pre-2023 data
            pre_2023_data = master_df.loc[master_df.index < '2023-01-01', col]
            missing_percentage = (pre_2023_data.isna().sum() / len(pre_2023_data)) * 100 if len(pre_2023_data) > 0 else 0
            
            # Keep column if it has recent data OR if pre-2023 missing data is less than threshold
            if has_recent_data or missing_percentage < 95.45:
                columns_to_keep.append(col)
        
        # Filter DataFrame to keep only selected columns
        master_df = master_df[columns_to_keep]
        
        print(f"Original columns: {original_columns}")
        print(f"Columns after filtering: {len(master_df.columns)}")
        print(f"Removed {original_columns - len(master_df.columns)} columns")
        
        # Ensure all data is at hourly frequency
        print("\nEnsuring hourly frequency for all data...")
        master_df = master_df.asfreq('h')
        
        # Save the combined DataFrame
        print(f"\nSaving master dataset...")
        print(f"Total columns: {len(master_df.columns)}")
        print(f"Date range: {master_df.index.min()} to {master_df.index.max()}")
        print(f"Total rows: {len(master_df)}")
        
        # Calculate memory usage
        memory_usage = master_df.memory_usage(deep=True).sum() / 1024**2  # Convert to MB
        print(f"Memory usage: {memory_usage:.2f} MB")
        
        # Save to CSV
        master_df.to_csv(output_path)
        print(f"\nMaster dataset saved to: {output_path}")
        
        # Generate summary statistics
        summary_path = Path("master_data/summary_master_data.csv")
        summary = pd.DataFrame({
            'non_null_count': master_df.count(),
            'null_count': master_df.isnull().sum(),
            'null_percentage': (master_df.isnull().sum() / len(master_df) * 100).round(2),
            'unique_values': master_df.nunique(),
            'memory_usage_mb': master_df.memory_usage(deep=True) / 1024**2
        })
        summary.to_csv(summary_path)
        print(f"Summary statistics saved to: {summary_path}")
        
    else:
        print("No data was processed. Please check the input directory and files.")
        

if __name__ == "__main__":
    combine_csv_files()

### Combine TO and FROM parameters into one, e.g. TO_SE3_FROM_X become TO_SE3 from all sources

In [ ]:
# Read the dataset
csv_path = Path("master_data/resultsset.csv")
df = pd.read_csv(csv_path)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)

# List of regions and generation types
regions = ['SE', 'SE1', 'SE2', 'SE3', 'SE4']
generation_types = ['Hydro Water Reservoir', 'Wind Onshore', 'Fossil Gas', 'Fossil Oil', 'Nuclear']

# 1. For Outages_UnavailabilityOfProductionUnits, keep only columns with space before underscore
for region in regions:
    for gen_type in generation_types:
        # Find columns with space before underscore
        space_cols = [col for col in df.columns if f'Outages_UnavailabilityOfProductionUnits_{region}_{gen_type} _' in col]
        # Find columns without space before underscore
        no_space_cols = [col for col in df.columns if f'Outages_UnavailabilityOfProductionUnits_{region}_{gen_type}_' in col 
                        and col not in space_cols]
        
        # Drop columns without space
        if no_space_cols:
            df = df.drop(columns=no_space_cols)

# 2. Remove Outages_UnavailabilityOfGenerationUnits columns
generation_cols = [col for col in df.columns if 'Outages_UnavailabilityOfGenerationUnits_' in col]
df = df.drop(columns=generation_cols)

# 3. Aggregate Outages_UnavailabilityInTransmissionGrid columns
for region in regions:
    # Find all FROM columns for this region (both where region is FROM and TO)
    from_cols = [col for col in df.columns if f'Outages_UnavailabilityInTransmissionGrid_FROM_{region}_TO_' in col]
    to_cols = [col for col in df.columns if f'Outages_UnavailabilityInTransmissionGrid_FROM_' in col and f'_TO_{region}_' in col]
    
    if from_cols:
        df[f'Outages_UnavailabilityInTransmissionGrid_FROM_{region}_NewNTC'] = df[from_cols].sum(axis=1)
        df = df.drop(columns=from_cols)
    
    if to_cols:
        df[f'Outages_UnavailabilityInTransmissionGrid_TO_{region}_NewNTC'] = df[to_cols].sum(axis=1)
        df = df.drop(columns=to_cols)

# 4. Aggregate Congestion Management columns
for region in regions:
    # Find all FROM columns for this region (both where region is FROM and TO)
    from_cols = [col for col in df.columns if f'Congestion Management_Countertrading_FROM_{region}_TO_' in col]
    to_cols = [col for col in df.columns if f'Congestion Management_Countertrading_FROM_' in col and f'_TO_{region}_' in col]
    
    if from_cols:
        df[f'Congestion Management_Countertrading_FROM_{region}_ChangeInCrosszonalExchange(MW)'] = df[from_cols].sum(axis=1)
        df = df.drop(columns=from_cols)
    
    if to_cols:
        df[f'Congestion Management_Countertrading_TO_{region}_ChangeInCrosszonalExchange(MW)'] = df[to_cols].sum(axis=1)
        df = df.drop(columns=to_cols)

# 5. Aggregate transmission capacity and flow columns
transmission_patterns = [
    ('Transmission_ForecastedYearAheadTransferCapacities_', 'ForecastTransferCapacity'),
    ('Transmission_ForecastedMonthAheadTransferCapacities_', 'ForecastTransferCapacity'),
    ('Transmission_ForecastedWeekAheadTransferCapacities_', 'ForecastTransferCapacity'),
    ('Transmission_OfferedIntradayTransferCapacityImplicit_', 'Capacity'),
    ('Transmission_PhysicalFlows_', 'FlowValue')
]

for pattern, suffix in transmission_patterns:
    for region in regions:
        # Find all FROM columns for this region (both where region is FROM and TO)
        from_cols = [col for col in df.columns if f'{pattern}FROM_{region}_TO_' in col]
        to_cols = [col for col in df.columns if f'{pattern}FROM_' in col and f'_TO_{region}_' in col]
        
        if from_cols:
            df[f'{pattern}FROM_{region}_{suffix}'] = df[from_cols].sum(axis=1)
            df = df.drop(columns=from_cols)
        
        if to_cols:
            df[f'{pattern}TO_{region}_{suffix}'] = df[to_cols].sum(axis=1)
            df = df.drop(columns=to_cols)

# 6. Aggregate Load forecasts
load_patterns = [
    'Load_WeekAheadTotalLoadForecast_',
    'Load_YearAheadTotalLoadForecast_',
    'Load_MonthAheadTotalLoadForecast_'
]

for pattern in load_patterns:
    for region in regions:
        min_col = f"{pattern}{region}_MinimumTotalLoadValue"
        max_col = f"{pattern}{region}_MaximumTotalLoadValue"
        
        if min_col in df.columns and max_col in df.columns:
            new_col = f"{pattern}{region}_AvgMinMax"
            df[new_col] = df[[min_col, max_col]].mean(axis=1)
            df = df.drop(columns=[min_col, max_col])


# Define the columns to keep
columns_to_keep = []

# Base patterns for each region
for region in regions:
    patterns = [
        f"Congestion Management_Countertrading_FROM_{region}_ChangeInCrosszonalExchange(MW)",
        f"Congestion Management_Countertrading_TO_{region}_ChangeInCrosszonalExchange(MW)",
        f"Outages_UnavailabilityInTransmissionGrid_FROM_{region}_NewNTC",
        f"Outages_UnavailabilityInTransmissionGrid_TO_{region}_NewNTC",
        f"Load_ActualTotalLoad_{region}_TotalLoadValue",
        f"Load_WeekAheadTotalLoadForecast_{region}_AvgMinMax",
        f"Load_YearAheadTotalLoadForecast_{region}_AvgMinMax",
        f"Load_MonthAheadTotalLoadForecast_{region}_AvgMinMax",
        f"Load_DayAheadTotalLoadForecast_{region}_TotalLoadValue",
        f"Balancing_ImbalancePrices_{region}_PositiveImbalancePrice",
        f"Balancing_ImbalancePrices_{region}_NegativeImbalancePrice",
        f"Balancing_AggregatedBalancingEnergyBids_{region}_mFRR_OfferedUpBidVolume[MW]",
        f"Balancing_AggregatedBalancingEnergyBids_{region}_mFRR_OfferedDownBidVolume[MW]",
        f"Balancing_AggregatedBalancingEnergyBids_{region}_mFRR_ActivatedDownBidVolume[MW]",
        f"Balancing_AggregatedBalancingEnergyBids_{region}_mFRR_ActivatedUpBidVolume[MW]",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_aFRR_Down_Volume(MW)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_aFRR_Up_Volume(MW)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_FCR_Symmetric_Volume(MW)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_mFRR_Down_Volume(MW)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_mFRR_Up_Volume(MW)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_aFRR_Down_Price(MW/ISP)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_aFRR_Up_Price(MW/ISP)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_FCR_Symmetric_Price(MW/ISP)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_mFRR_Down_Price(MW/ISP)",
        f"Balancing_AmountAndPricesPaidOfBalancingReservesUnderContract_{region}_mFRR_Up_Price(MW/ISP)",
        f"Balancing_PricesOfActivatedBalancingEnergy_{region}_aFRR_NotSpecifiedUpPrice",
        f"Balancing_PricesOfActivatedBalancingEnergy_{region}_FCR_NotSpecifiedUpPrice",
        f"Balancing_PricesOfActivatedBalancingEnergy_{region}_mFRR_NotSpecifiedUpPrice",
        f"Balancing_PricesOfActivatedBalancingEnergy_{region}_aFRR_NotSpecifiedDownPrice",
        f"Balancing_PricesOfActivatedBalancingEnergy_{region}_FCR_NotSpecifiedDownPrice",
        f"Balancing_PricesOfActivatedBalancingEnergy_{region}_mFRR_NotSpecifiedDownPrice",
        f"Balancing_ActivatedBalancingEnergy_{region}_aFRR_NotSpecifiedUpActivatedVolume",
        f"Balancing_ActivatedBalancingEnergy_{region}_FCR_NotSpecifiedUpActivatedVolume",
        f"Balancing_ActivatedBalancingEnergy_{region}_mFRR_NotSpecifiedUpActivatedVolume",
        f"Balancing_ActivatedBalancingEnergy_{region}_aFRR_NotSpecifiedDownActivatedVolume",
        f"Balancing_ActivatedBalancingEnergy_{region}_FCR_NotSpecifiedDownActivatedVolume",
        f"Balancing_ActivatedBalancingEnergy_{region}_mFRR_NotSpecifiedDownActivatedVolume",
        f"Balancing_AcceptedAggregatedOffers_{region}_aFRR_NotSpecifiedUpAcceptedVolume",
        f"Balancing_AcceptedAggregatedOffers_{region}_mFRR_NotSpecifiedUpAcceptedVolume",
        f"Balancing_AcceptedAggregatedOffers_{region}_aFRR_NotSpecifiedDownAcceptedVolume",
        f"Balancing_AcceptedAggregatedOffers_{region}_mFRR_NotSpecifiedDownAcceptedVolume",
        f"Transmission_ImplicitAllocationsNetPositions_{region}_Daily_Export_NetPosition[MW]",
        f"Transmission_ImplicitAllocationsNetPositions_{region}_Daily_Import_NetPosition[MW]",
        f"Transmission_ForecastedYearAheadTransferCapacities_TO_{region}_ForecastTransferCapacity",
        f"Transmission_ForecastedYearAheadTransferCapacities_FROM_{region}_ForecastTransferCapacity",
        f"Transmission_ForecastedMonthAheadTransferCapacities_TO_{region}_ForecastTransferCapacity",
        f"Transmission_ForecastedMonthAheadTransferCapacities_FROM_{region}_ForecastTransferCapacity",
        f"Transmission_ForecastedWeekAheadTransferCapacities_TO_{region}_ForecastTransferCapacity",
        f"Transmission_ForecastedWeekAheadTransferCapacities_FROM_{region}_ForecastTransferCapacity",
        f"Transmission_OfferedIntradayTransferCapacityImplicit_TO_{region}_Capacity",
        f"Transmission_OfferedIntradayTransferCapacityImplicit_FROM_{region}_Capacity",
        f"Transmission_PhysicalFlows_TO_{region}_FlowValue",
        f"Transmission_PhysicalFlows_FROM_{region}_FlowValue",
        f"Transmission_EnergyPrices_{region}_Price[Currency/MWh]",
        f"Generation_CurrentGenerationForecastForWindAndSolar_{region}_AggregatedGenerationForecast",
        f"Generation_AggregatedFillingRateOfWaterReservoirsAndHydroStoragePlants_{region}_StoredEnergy",
        f"Generation_DayAheadAggregatedGeneration_{region}_ScheduledGeneration",
        f"Generation_DayAheadGenerationForecastForWindAndSolar_{region}_AggregatedGenerationForecast"
    ]
    columns_to_keep.extend(patterns)
    
    # Add generation type specific columns
    for gen_type in generation_types:
        columns_to_keep.append(f"Outages_UnavailabilityOfProductionUnits_{region}_{gen_type} _UnavailableCapacity")

# Filter columns that actually exist in the DataFrame
existing_columns = [col for col in columns_to_keep if col in df.columns]

# Keep only the specified columns
df = df[existing_columns]

# Print some information about the filtering
print(f"Original number of columns requested: {len(columns_to_keep)}")
print(f"Number of columns found in dataset: {len(existing_columns)}")
print(f"Missing columns: {len(columns_to_keep) - len(existing_columns)}")

# Save the modified dataset
save_path = Path("master_data/results_final.csv")
df.to_csv(save_path)

### Clean up column names

In [ ]:
import pandas as pd

# Path to the input CSV file
input_file = Path("master_data/results_final.csv")

# Path to the output CSV file
output_file = Path("master_data/results_final_fixed.csv")

# Read the CSV file
print(f"Reading CSV file: {input_file}")
df = pd.read_csv(input_file)

# Get the original column names
original_columns = df.columns.tolist()
print(f"Number of columns in CSV: {len(original_columns)}")

# Create new column names by removing spaces
new_columns = [col.replace(' ', '') for col in original_columns]

# Rename the columns
df.columns = new_columns

# Save the dataframe to a new CSV file
print(f"Saving to: {output_file}")
df.to_csv(output_file, index=False)

print("Column renaming complete!")
print("Examples of renamed columns:")
for i, (old, new) in enumerate(zip(original_columns, new_columns)):
    if old != new:
        print(f"  {old} -> {new}")
    if i >= 5:  # Just show a few examples
        print("  ...")
        break



In [ ]:
import numpy as np

# Path to the input CSV file
input_file = Path("master_data/results_final.csv")

# Path to the output CSV file
output_file = Path("master_data/results_final_fixed.csv")

def verify_files_equal():
    """
    Verify that the content of both files is identical except for the headers.
    """
    print("\nVerifying files have identical content...")
    
    # Read both files
    original_df = pd.read_csv(input_file)
    new_df = pd.read_csv(output_file)
    
    # Check number of rows and columns
    if original_df.shape != new_df.shape:
        print(f" Files have different shapes: {original_df.shape} vs {new_df.shape}")
        return False
    
    # Rename columns in original to match new format
    renamed_original_df = original_df.copy()
    renamed_original_df.columns = [col.replace(' ', '') for col in original_df.columns]
    
    # Verify all renamed columns match the new file's columns
    col_match = all(a == b for a, b in zip(renamed_original_df.columns, new_df.columns))
    if not col_match:
        print(" Renamed columns don't match exactly")
        # Print columns that don't match
        for i, (a, b) in enumerate(zip(renamed_original_df.columns, new_df.columns)):
            if a != b:
                print(f"  Column {i}: '{a}' vs '{b}'")
        return False
    
    # Compare data values column by column
    all_equal = True
    for col in renamed_original_df.columns:
        if not renamed_original_df[col].equals(new_df[col]):
            all_equal = False
            print(f" Column '{col}' has different values")
            
            # Show a sample of differences
            diff_indices = ~(renamed_original_df[col] == new_df[col])
            if diff_indices.any():
                diff_rows = diff_indices[diff_indices].index[:5]  # Get first 5 different rows
                print("  Sample differences (first 5):")
                for idx in diff_rows:
                    print(f"  Row {idx}: '{renamed_original_df.loc[idx, col]}' vs '{new_df.loc[idx, col]}'")
    
    # Check for any NA values that might differ
    original_na_count = renamed_original_df.isna().sum().sum()
    new_na_count = new_df.isna().sum().sum()
    if original_na_count != new_na_count:
        all_equal = False
        print(f" Number of NA values differs: {original_na_count} in original vs {new_na_count} in new file")
        
        # Check which columns have different NA counts
        for col in renamed_original_df.columns:
            orig_na = renamed_original_df[col].isna().sum()
            new_na = new_df[col].isna().sum()
            if orig_na != new_na:
                print(f"  Column '{col}' has different NA counts: {orig_na} vs {new_na}")
    
    if all_equal:
        print("Verification successful! Files have identical data with only header differences")
        return True
    else:
        return False

# Call the verification function after creating the new file
verify_files_equal()

### Merge with other master data to extend data

In [30]:
import pandas as pd
from pathlib import Path

# paths to your files
old_path   = Path("processed_data/results/resultsset_v4.csv")
new_path   = Path("master_data/results_final_fixed.csv")
save_path  = Path("master_data/results_merged.csv")

# 1) Load both CSVs, parsing the datetime index
old = pd.read_csv(old_path, parse_dates=True, index_col=0)
new = pd.read_csv(new_path, parse_dates=True, index_col=0)
old.index = pd.to_datetime(old.index)
new.index = pd.to_datetime(new.index)

# 2) Find which columns they share
common_cols = old.columns.intersection(new.columns)

# 3) Determine the cut‐off timestamp in the old data
cutoff_old = old.index.max()

# 4) Pull only those rows & columns from the new data beyond the old cutoff
new_beyond = new.loc[new.index > cutoff_old, common_cols]

# 5) Concatenate, sort, then truncate at 2025-05-01
merged = pd.concat([old, new_beyond], axis=0).sort_index()

# --- cut at ---
merged = merged.loc[:'2025-02-20']

# 6) Write out your merged file
merged.to_csv(save_path)


In [ ]:
csv  = Path("master_data/results_merged.csv")
df = pd.read_csv(csv)

df.head()